In [30]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cicids2017/Benign-Monday-no-metadata.parquet
/kaggle/input/cicids2017/Bruteforce-Tuesday-no-metadata.parquet
/kaggle/input/cicids2017/Portscan-Friday-no-metadata.parquet
/kaggle/input/cicids2017/WebAttacks-Thursday-no-metadata.parquet
/kaggle/input/cicids2017/DoS-Wednesday-no-metadata.parquet
/kaggle/input/cicids2017/DDoS-Friday-no-metadata.parquet
/kaggle/input/cicids2017/Infiltration-Thursday-no-metadata.parquet
/kaggle/input/cicids2017/Botnet-Friday-no-metadata.parquet
/kaggle/input/cleaned-ids/ids2018_cleaned_combined_1.csv
/kaggle/input/ids-intrusion-csv/02-28-2018.csv
/kaggle/input/ids-intrusion-csv/03-01-2018.csv
/kaggle/input/ids-intrusion-csv/02-16-2018.csv
/kaggle/input/ids-intrusion-csv/02-15-2018.csv
/kaggle/input/ids-intrusion-csv/02-21-2018.csv
/kaggle/input/ids-intrusion-csv/03-02-2018.csv
/kaggle/input/ids-intrusion-csv/02-22-2018.csv
/kaggle/input/ids-intrusion-csv/02-20-2018.csv
/kaggle/input/ids-intrusion-csv/02-14-2018.csv
/kaggle/input/ids-intrusion-c

In [31]:
print("hi")

hi


In [32]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# 1️⃣ Load dataset
df = pd.read_csv("/kaggle/input/traininig/UNSW_NB15_training-set.csv")

# 2️⃣ Remove columns where all values are NaN
df = df.dropna(axis=1, how='all')

# 3️⃣ Remove columns where all values are 0
df = df.loc[:, (df != 0).any(axis=0)]

# 4️⃣ Remove duplicate rows
df = df.drop_duplicates()

# 5️⃣ Identify column types
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
cat_cols = df.select_dtypes(include=['object']).columns

# 6️⃣ Handle missing values
df[num_cols] = df[num_cols].fillna(df[num_cols].median())
df[cat_cols] = df[cat_cols].fillna(df[cat_cols].mode().iloc[0])

# 7️⃣ Label encode categorical columns
le = LabelEncoder()
for col in cat_cols:
    df[col] = le.fit_transform(df[col].astype(str))

# 8️⃣ Min-Max normalization
scaler = MinMaxScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

# 9️⃣ Save final output in Kaggle working directory
output_filename = "/kaggle/working/network_train_final_normalized.csv"
df.to_csv(output_filename, index=False)

print("✅ Preprocessing complete!")
print("📁 Saved as:", output_filename)
print("Shape:", df.shape)

KeyboardInterrupt: 

In [ ]:
# catboost_training_label_target_fixed.py

import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, classification_report, confusion_matrix)

def detect_categorical(df, threshold_unique_ratio=0.05, max_unique_for_cat=50):
    cats = []
    n = len(df)
    for col in df.columns:
        if df[col].dtype == 'object' or pd.api.types.is_categorical_dtype(df[col]):
            cats.append(col)
        else:
            nunq = df[col].nunique(dropna=True)
            if nunq <= max_unique_for_cat and (nunq / max(1, n)) <= threshold_unique_ratio:
                cats.append(col)
    return cats

def main(csv_path="/kaggle/working/network_train_final_normalized.csv",
         test_size=0.2,
         random_state=42):

    print("Loading dataset:", csv_path)
    df = pd.read_csv(csv_path)
    print("Shape:", df.shape)
    print(df.head())

    # fixed target and leakage columns
    target_col = "label"
    leak_cols = ["attack_cat"]  # remove leakage column(s)

    # checks
    missing = [c for c in [target_col] + leak_cols if c not in df.columns]
    if missing:
        raise ValueError(f"Missing expected column(s): {missing}")

    print("Using target column:", target_col)
    print("Removing leakage column(s):", leak_cols)

    # drop target & leakage from features
    X = df.drop(columns=[target_col] + leak_cols)
    y = df[target_col].copy()

    # convert non-numeric target to numeric (if needed)
    if not pd.api.types.is_numeric_dtype(y):
        print("Target is non-numeric — applying label encoding.")
        y = pd.Series(pd.factorize(y)[0], index=y.index)

    # detect categorical features in X
    cat_cols = detect_categorical(X)
    # remove any accidental numeric-id-like high-cardinality columns from cat list
    refined_cat_cols = []
    for c in cat_cols:
        nunq = X[c].nunique(dropna=True)
        if nunq <= min(100, max(10, int(0.5 * len(X)))):
            refined_cat_cols.append(c)
    cat_cols = refined_cat_cols

    print("Categorical columns to pass to CatBoost:", cat_cols)

    # convert categorical columns to string (CatBoost can accept them)
    for c in cat_cols:
        X[c] = X[c].astype(str).fillna("##nan##")  

    # train-test split
    stratify = y if len(np.unique(y)) > 1 else None
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=stratify
    )

    cat_indices = [X.columns.get_loc(c) for c in cat_cols if c in X.columns]

    train_pool = Pool(X_train, y_train, cat_features=cat_indices)
    test_pool = Pool(X_test, y_test, cat_features=cat_indices)

    n_classes = len(np.unique(y_train))
    is_binary = (n_classes == 2)

    model = CatBoostClassifier(
        iterations=700,
        learning_rate=0.05,
        depth=6,
        eval_metric='AUC' if is_binary else 'MultiClass',
        loss_function='Logloss' if is_binary else 'MultiClass',
        random_seed=random_state,
        early_stopping_rounds=50,
        verbose=100
    )

    print("\nTraining CatBoost...")
    model.fit(train_pool, eval_set=test_pool, use_best_model=True)

    preds = model.predict(X_test)
    if isinstance(preds, np.ndarray) and preds.ndim > 1:
        preds = preds.flatten()

    try:
        pred_proba = model.predict_proba(X_test)
    except Exception:
        pred_proba = None

    print("\n===== METRICS =====")
    print("Accuracy:", accuracy_score(y_test, preds))

    if is_binary:
        print("Precision:", precision_score(y_test, preds, zero_division=0))
        print("Recall:", recall_score(y_test, preds, zero_division=0))
        print("F1 Score:", f1_score(y_test, preds, zero_division=0))
        if pred_proba is not None:
            try:
                print("ROC AUC:", roc_auc_score(y_test, pred_proba[:, 1]))
            except Exception as e:
                print("ROC AUC could not be computed:", e)
    else:
        print("Macro Precision:", precision_score(y_test, preds, average="macro", zero_division=0))
        print("Macro Recall:", recall_score(y_test, preds, average="macro", zero_division=0))
        print("Macro F1:", f1_score(y_test, preds, average="macro", zero_division=0))

    print("\nClassification Report:\n", classification_report(y_test, preds, zero_division=0))
    print("Confusion Matrix:\n", confusion_matrix(y_test, preds))

    # optionally save model
    # model.save_model("/kaggle/working/catboost_label_model.cbm")

    return model

if __name__ == "__main__":
    model = main()


In [ ]:
df = pd.read_csv("/kaggle/input/networkkk/loan_final_normalized (1).csv")

In [ ]:
df.columns

In [ ]:
(df["attack_cat"] == 0).astype(int).value_counts()
df.groupby("attack_cat")["label"].unique()


In [ ]:
df.head(5)

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dhoogla/cicids2017")

print("Path to dataset files:", path)

In [ ]:
import pandas as pd

# Load the Parquet file
file_path = "/kaggle/input/cicids2017/Benign-Monday-no-metadata.parquet"
df = pd.read_parquet(file_path)

# Shape: rows & columns
print("Number of rows:", df.shape[0])
print("Number of columns:", df.shape[1])

# Column names
print("\nColumn names:")
print(df.columns.tolist())

In [ ]:
# Unique values in the label column
label_col = "Label"   # change if the column name differs
if label_col in df.columns:
    print("\nUnique values in Label column:")
    print(df[label_col].unique())
else:
    print("\nNo 'Label' column found. Available columns:")
    print(df.columns.tolist())

In [43]:
import pandas as pd

# Load the Parquet file
file_path = "/kaggle/input/cicids2017/Botnet-Friday-no-metadata.parquet"
df = pd.read_parquet(file_path)

# Shape: rows & columns
print("Number of rows:", df.shape[0])
print("Number of columns:", df.shape[1])

# Column names
print("\nColumn names:")
print(df.columns.tolist())

Number of rows: 176038
Number of columns: 78

Column names:
['Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Fwd Packets Length Total', 'Bwd Packets Length Total', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG 

In [44]:
# Unique values in the label column
label_col = "Label"   # change if the column name differs
if label_col in df.columns:
    print("\nUnique values in Label column:")
    print(df[label_col].unique())
else:
    print("\nNo 'Label' column found. Available columns:")
    print(df.columns.tolist())


Unique values in Label column:
['Benign', 'Bot']
Categories (2, object): ['Benign', 'Bot']


In [45]:
# Unique values in the label column + their counts
label_col = "Label"   # change if needed

if label_col in df.columns:
    print("\nUnique values and counts in Label column:")
    print(df[label_col].value_counts())
else:
    print("\nNo 'Label' column found. Available columns:")
    print(df.columns.tolist())



Unique values and counts in Label column:
Label
Benign    174601
Bot         1437
Name: count, dtype: int64


In [46]:
import pandas as pd

# Load the Parquet file
file_path = "/kaggle/input/cicids2017/Bruteforce-Tuesday-no-metadata.parquet"
df = pd.read_parquet(file_path)

# Shape: rows & columns
print("Number of rows:", df.shape[0])
print("Number of columns:", df.shape[1])

# Column names
print("\nColumn names:")
print(df.columns.tolist())

Number of rows: 389714
Number of columns: 78

Column names:
['Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Fwd Packets Length Total', 'Bwd Packets Length Total', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG 

In [47]:
# Unique values in the label column + their counts
label_col = "Label"   # change if needed

if label_col in df.columns:
    print("\nUnique values and counts in Label column:")
    print(df[label_col].value_counts())
else:
    print("\nNo 'Label' column found. Available columns:")
    print(df.columns.tolist())



Unique values and counts in Label column:
Label
Benign         380564
FTP-Patator      5931
SSH-Patator      3219
Name: count, dtype: int64


In [48]:
import pandas as pd

# Load the Parquet file
file_path = "/kaggle/input/cicids2017/DDoS-Friday-no-metadata.parquet"
df = pd.read_parquet(file_path)

# Shape: rows & columns
print("Number of rows:", df.shape[0])
print("Number of columns:", df.shape[1])

# Column names
print("\nColumn names:")
print(df.columns.tolist())

Number of rows: 221264
Number of columns: 78

Column names:
['Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Fwd Packets Length Total', 'Bwd Packets Length Total', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG 

In [49]:
# Unique values in the label column + their counts
label_col = "Label"   # change if needed

if label_col in df.columns:
    print("\nUnique values and counts in Label column:")
    print(df[label_col].value_counts())
else:
    print("\nNo 'Label' column found. Available columns:")
    print(df.columns.tolist())



Unique values and counts in Label column:
Label
DDoS      128014
Benign     93250
Name: count, dtype: int64


In [50]:
import pandas as pd

# Load the Parquet file
file_path = "/kaggle/input/cicids2017/DoS-Wednesday-no-metadata.parquet"
df = pd.read_parquet(file_path)

# Shape: rows & columns
print("Number of rows:", df.shape[0])
print("Number of columns:", df.shape[1])

# Column names
print("\nColumn names:")
print(df.columns.tolist())

Number of rows: 584991
Number of columns: 78

Column names:
['Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Fwd Packets Length Total', 'Bwd Packets Length Total', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG 

In [51]:
# Unique values in the label column + their counts
label_col = "Label"   # change if needed

if label_col in df.columns:
    print("\nUnique values and counts in Label column:")
    print(df[label_col].value_counts())
else:
    print("\nNo 'Label' column found. Available columns:")
    print(df.columns.tolist())



Unique values and counts in Label column:
Label
Benign              391235
DoS Hulk            172846
DoS GoldenEye        10286
DoS slowloris         5385
DoS Slowhttptest      5228
Heartbleed              11
Name: count, dtype: int64


In [52]:
import pandas as pd

# Load the Parquet file
file_path = "/kaggle/input/cicids2017/Infiltration-Thursday-no-metadata.parquet"
df = pd.read_parquet(file_path)

# Shape: rows & columns
print("Number of rows:", df.shape[0])
print("Number of columns:", df.shape[1])

# Column names
print("\nColumn names:")
print(df.columns.tolist())

Number of rows: 207630
Number of columns: 78

Column names:
['Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Fwd Packets Length Total', 'Bwd Packets Length Total', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG 

In [53]:
# Unique values in the label column + their counts
label_col = "Label"   # change if needed

if label_col in df.columns:
    print("\nUnique values and counts in Label column:")
    print(df[label_col].value_counts())
else:
    print("\nNo 'Label' column found. Available columns:")
    print(df.columns.tolist())



Unique values and counts in Label column:
Label
Benign          207594
Infiltration        36
Name: count, dtype: int64


In [54]:
import pandas as pd

# Load the Parquet file
file_path = "/kaggle/input/cicids2017/Portscan-Friday-no-metadata.parquet"
df = pd.read_parquet(file_path)

# Shape: rows & columns
print("Number of rows:", df.shape[0])
print("Number of columns:", df.shape[1])

# Column names
print("\nColumn names:")
print(df.columns.tolist())

Number of rows: 119522
Number of columns: 78

Column names:
['Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Fwd Packets Length Total', 'Bwd Packets Length Total', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG 

In [55]:
# Unique values in the label column + their counts
label_col = "Label"   # change if needed

if label_col in df.columns:
    print("\nUnique values and counts in Label column:")
    print(df[label_col].value_counts())
else:
    print("\nNo 'Label' column found. Available columns:")
    print(df.columns.tolist())



Unique values and counts in Label column:
Label
Benign      117566
PortScan      1956
Name: count, dtype: int64


In [56]:
import pandas as pd

# Load the Parquet file
file_path = "/kaggle/input/cicids2017/WebAttacks-Thursday-no-metadata.parquet"
df = pd.read_parquet(file_path)

# Shape: rows & columns
print("Number of rows:", df.shape[0])
print("Number of columns:", df.shape[1])

# Column names
print("\nColumn names:")
print(df.columns.tolist())

Number of rows: 155820
Number of columns: 78

Column names:
['Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Fwd Packets Length Total', 'Bwd Packets Length Total', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG 

In [57]:
# Unique values in the label column + their counts
label_col = "Label"   # change if needed

if label_col in df.columns:
    print("\nUnique values and counts in Label column:")
    print(df[label_col].value_counts())
else:
    print("\nNo 'Label' column found. Available columns:")
    print(df.columns.tolist())



Unique values and counts in Label column:
Label
Benign                        153677
Web Attack � Brute Force        1470
Web Attack � XSS                 652
Web Attack � Sql Injection        21
Name: count, dtype: int64


IDS 2018

In [58]:
import pandas as pd

# Load the Parquet file
file_path = "/kaggle/input/ids-intrusion-csv/02-14-2018.csv"
df = pd.read_csv(file_path)

# Shape: rows & columns
print("Number of rows:", df.shape[0])
print("Number of columns:", df.shape[1])

# Column names
print("\nColumn names:")
print(df.columns.tolist())

Number of rows: 1048575
Number of columns: 80

Column names:
['Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'F

In [59]:
# Unique values in the label column + their counts
label_col = "Label"   # change if needed

if label_col in df.columns:
    print("\nUnique values and counts in Label column:")
    print(df[label_col].value_counts())
else:
    print("\nNo 'Label' column found. Available columns:")
    print(df.columns.tolist())



Unique values and counts in Label column:
Label
Benign            667626
FTP-BruteForce    193360
SSH-Bruteforce    187589
Name: count, dtype: int64


BRUTE FORCE IDS

In [60]:
import pandas as pd

# Load the Parquet file
file_path = "/kaggle/input/ids-intrusion-csv/02-15-2018.csv"
df = pd.read_csv(file_path)

# Shape: rows & columns
print("Number of rows:", df.shape[0])
print("Number of columns:", df.shape[1])

# Column names
print("\nColumn names:")
print(df.columns.tolist())

Number of rows: 1048575
Number of columns: 80

Column names:
['Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'F

In [61]:
# Unique values in the label column + their counts
label_col = "Label"   # change if needed

if label_col in df.columns:
    print("\nUnique values and counts in Label column:")
    print(df[label_col].value_counts())
else:
    print("\nNo 'Label' column found. Available columns:")
    print(df.columns.tolist())



Unique values and counts in Label column:
Label
Benign                   996077
DoS attacks-GoldenEye     41508
DoS attacks-Slowloris     10990
Name: count, dtype: int64


Dos attack

In [62]:
import pandas as pd

# Load the Parquet file
file_path = "/kaggle/input/ids-intrusion-csv/02-16-2018.csv"
df = pd.read_csv(file_path)

# Shape: rows & columns
print("Number of rows:", df.shape[0])
print("Number of columns:", df.shape[1])

# Column names
print("\nColumn names:")
print(df.columns.tolist())

/tmp/ipykernel_47/1221501978.py:5: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Number of rows: 1048575
Number of columns: 80

Column names:
['Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'F

In [63]:
# Unique values in the label column + their counts
label_col = "Label"   # change if needed

if label_col in df.columns:
    print("\nUnique values and counts in Label column:")
    print(df[label_col].value_counts())
else:
    print("\nNo 'Label' column found. Available columns:")
    print(df.columns.tolist())



Unique values and counts in Label column:
Label
DoS attacks-Hulk            461912
Benign                      446772
DoS attacks-SlowHTTPTest    139890
Label                            1
Name: count, dtype: int64


Dos

In [64]:
import pandas as pd

# Load the Parquet file
file_path = "/kaggle/input/ids-intrusion-csv/02-20-2018.csv"
df = pd.read_csv(file_path)

# Shape: rows & columns
print("Number of rows:", df.shape[0])
print("Number of columns:", df.shape[1])

# Column names
print("\nColumn names:")
print(df.columns.tolist())

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
# Unique values in the label column + their counts
label_col = "Label"   # change if needed

if label_col in df.columns:
    print("\nUnique values and counts in Label column:")
    print(df[label_col].value_counts())
else:
    print("\nNo 'Label' column found. Available columns:")
    print(df.columns.tolist())


In [ ]:
import pandas as pd

# Load the Parquet file
file_path = "/kaggle/input/ids-intrusion-csv/02-21-2018.csv"
df = pd.read_csv(file_path)

# Shape: rows & columns
print("Number of rows:", df.shape[0])
print("Number of columns:", df.shape[1])

# Column names
print("\nColumn names:")
print(df.columns.tolist())

In [ ]:
# Unique values in the label column + their counts
label_col = "Label"   # change if needed

if label_col in df.columns:
    print("\nUnique values and counts in Label column:")
    print(df[label_col].value_counts())
else:
    print("\nNo 'Label' column found. Available columns:")
    print(df.columns.tolist())


In [ ]:
import pandas as pd

# Load the Parquet file
file_path = "/kaggle/input/ids-intrusion-csv/02-22-2018.csv"
df = pd.read_csv(file_path)

# Shape: rows & columns
print("Number of rows:", df.shape[0])
print("Number of columns:", df.shape[1])

# Column names
print("\nColumn names:")
print(df.columns.tolist())

In [ ]:
# Unique values in the label column + their counts
label_col = "Label"   # change if needed

if label_col in df.columns:
    print("\nUnique values and counts in Label column:")
    print(df[label_col].value_counts())
else:
    print("\nNo 'Label' column found. Available columns:")
    print(df.columns.tolist())


In [ ]:
import pandas as pd

# Load the Parquet file
file_path = "/kaggle/input/ids-intrusion-csv/02-23-2018.csv"
df = pd.read_csv(file_path)

# Shape: rows & columns
print("Number of rows:", df.shape[0])
print("Number of columns:", df.shape[1])

# Column names
print("\nColumn names:")
print(df.columns.tolist())

In [ ]:
# Unique values in the label column + their counts
label_col = "Label"   # change if needed

if label_col in df.columns:
    print("\nUnique values and counts in Label column:")
    print(df[label_col].value_counts())
else:
    print("\nNo 'Label' column found. Available columns:")
    print(df.columns.tolist())


In [ ]:
import pandas as pd

# Load the Parquet file
file_path = "/kaggle/input/ids-intrusion-csv/02-28-2018.csv"
df = pd.read_csv(file_path)

# Shape: rows & columns
print("Number of rows:", df.shape[0])
print("Number of columns:", df.shape[1])

# Column names
print("\nColumn names:")
print(df.columns.tolist())

In [ ]:
# Unique values in the label column + their counts
label_col = "Label"   # change if needed

if label_col in df.columns:
    print("\nUnique values and counts in Label column:")
    print(df[label_col].value_counts())
else:
    print("\nNo 'Label' column found. Available columns:")
    print(df.columns.tolist())


In [ ]:
import pandas as pd

# Load the Parquet file
file_path = "/kaggle/input/ids-intrusion-csv/03-01-2018.csv"
df = pd.read_csv(file_path)

# Shape: rows & columns
print("Number of rows:", df.shape[0])
print("Number of columns:", df.shape[1])

# Column names
print("\nColumn names:")
print(df.columns.tolist())

In [ ]:
# Unique values in the label column + their counts
label_col = "Label"   # change if needed

if label_col in df.columns:
    print("\nUnique values and counts in Label column:")
    print(df[label_col].value_counts())
else:
    print("\nNo 'Label' column found. Available columns:")
    print(df.columns.tolist())


In [ ]:
import pandas as pd

# Load the Parquet file
file_path = "/kaggle/input/ids-intrusion-csv/03-02-2018.csv"
df = pd.read_csv(file_path)

# Shape: rows & columns
print("Number of rows:", df.shape[0])
print("Number of columns:", df.shape[1])

# Column names
print("\nColumn names:")
print(df.columns.tolist())

In [ ]:
# Unique values in the label column + their counts
label_col = "Label"   # change if needed

if label_col in df.columns:
    print("\nUnique values and counts in Label column:")
    print(df[label_col].value_counts())
else:
    print("\nNo 'Label' column found. Available columns:")
    print(df.columns.tolist())


In [ ]:
import pandas as pd

# Load the full dataset
file_path1 = "/kaggle/input/ids-intrusion-csv/03-02-2018.csv"
df = pd.read_csv(file_path1)

file_path2 = "/kaggle/input/ids-intrusion-csv/03-01-2018.csv"
df2 = pd.read_csv(file_path2)

file_path3 = "/kaggle/input/ids-intrusion-csv/02-23-2018.csv"
df3 = pd.read_csv(file_path3)

file_path4 = "/kaggle/input/ids-intrusion-csv/02-21-2018.csv"
df4 = pd.read_csv(file_path4)

file_path5 = "/kaggle/input/ids-intrusion-csv/02-20-2018.csv"
df5 = pd.read_csv(file_path5)

file_path6 = "/kaggle/input/ids-intrusion-csv/02-16-2018.csv"
df6 = pd.read_csv(file_path6)

file_path7 = "/kaggle/input/ids-intrusion-csv/02-15-2018.csv"
df7 = pd.read_csv(file_path7)

file_path8 = "/kaggle/input/ids-intrusion-csv/02-14-2018.csv"
df8 = pd.read_csv(file_path8)

bot_df = df[df['Label'] == 'Bot'].sample(n=4771, random_state=42)

benign_df = df[df['Label'] == 'Benign'].sample(n=50000, random_state=42)

infiltration_df = df2[df2['Label'] == 'Infilteration'].sample(n=4771, random_state=42)

Brute_force_web_df = df3[df3['Label'] == 'Brute Force -Web'].sample(n=362, random_state=42)

Brute_force_xss_df = df3[df3['Label'] == 'Brute Force -XSS'].sample(n=151, random_state=42)

sql_injection_df = df3[df3['Label'] == 'SQL Injection'].sample(n=53, random_state=42)

ddos_hoic_df = df4[df4['Label'] == 'DDOS attack-HOIC'].sample(n=4771, random_state=42)

ddos_loicudp_df = df4[df4['Label'] == 'DDOS attack-LOIC-UDP'].sample(n=1730, random_state=42)

ddos_loichttp_df = df5[df5['Label'] == 'DDoS attacks-LOIC-HTTP'].sample(n=4771, random_state=42)

ddos_hulk_df = df6[df6['Label'] == 'DoS attacks-Hulk'].sample(n=4770, random_state=42)

ddos_slow_df = df6[df6['Label'] == 'DoS attacks-SlowHTTPTest'].sample(n=4770, random_state=42)

ddos_goldeye_df = df7[df7['Label'] == 'DoS attacks-GoldenEye'].sample(n=4770, random_state=42)

ddos_slowloris_df = df7[df7['Label'] == 'DoS attacks-Slowloris'].sample(n=4770, random_state=42)

ddos_ftp_bruteforce_df = df8[df8['Label'] == 'FTP-BruteForce'].sample(n=4770, random_state=42)

ddos_ssh_bruteforce_df = df8[df8['Label'] == 'SSH-Bruteforce'].sample(n=4770, random_state=42)

# --- 3. Combine both samples ---
combined_df = pd.concat([bot_df, benign_df], ignore_index=True)

# --- 4. Save to CSV ---
output_path = "/kaggle/working/bot_benign_sample.csv"
combined_df.to_csv(output_path, index=False)

print("Saved file:", output_path)
print("Final shape:", combined_df.shape)
print("Counts:\n", combined_df['Label'].value_counts())


In [ ]:
import pandas as pd

# ---------------- Load all datasets ----------------
df1 = pd.read_csv("/kaggle/input/ids-intrusion-csv/03-02-2018.csv")
df2 = pd.read_csv("/kaggle/input/ids-intrusion-csv/03-01-2018.csv")
df3 = pd.read_csv("/kaggle/input/ids-intrusion-csv/02-23-2018.csv")
df4 = pd.read_csv("/kaggle/input/ids-intrusion-csv/02-21-2018.csv")
df5 = pd.read_csv("/kaggle/input/ids-intrusion-csv/02-20-2018.csv")
df6 = pd.read_csv("/kaggle/input/ids-intrusion-csv/02-16-2018.csv")
df7 = pd.read_csv("/kaggle/input/ids-intrusion-csv/02-15-2018.csv")
df8 = pd.read_csv("/kaggle/input/ids-intrusion-csv/02-14-2018.csv")

# ---------------- Sampling ----------------

bot_df = df1[df1['Label'] == 'Bot'].sample(n=4771, random_state=42)
benign_df = df1[df1['Label'] == 'Benign'].sample(n=50000, random_state=42)

infiltration_df = df2[df2['Label'] == 'Infilteration'].sample(n=4771, random_state=42)

brute_web_df = df3[df3['Label'] == 'Brute Force -Web'].sample(n=362, random_state=42)
brute_xss_df = df3[df3['Label'] == 'Brute Force -XSS'].sample(n=151, random_state=42)
sql_injection_df = df3[df3['Label'] == 'SQL Injection'].sample(n=53, random_state=42)

ddos_hoic_df = df4[df4['Label'] == 'DDOS attack-HOIC'].sample(n=4771, random_state=42)
ddos_loicudp_df = df4[df4['Label'] == 'DDOS attack-LOIC-UDP'].sample(n=1730, random_state=42)

ddos_loichttp_df = df5[df5['Label'] == 'DDoS attacks-LOIC-HTTP'].sample(n=4771, random_state=42)

dos_hulk_df = df6[df6['Label'] == 'DoS attacks-Hulk'].sample(n=4770, random_state=42)
dos_slowhttp_df = df6[df6['Label'] == 'DoS attacks-SlowHTTPTest'].sample(n=4770, random_state=42)

dos_goldeneye_df = df7[df7['Label'] == 'DoS attacks-GoldenEye'].sample(n=4770, random_state=42)
dos_slowloris_df = df7[df7['Label'] == 'DoS attacks-Slowloris'].sample(n=4770, random_state=42)

ftp_bruteforce_df = df8[df8['Label'] == 'FTP-BruteForce'].sample(n=4770, random_state=42)
ssh_bruteforce_df = df8[df8['Label'] == 'SSH-Bruteforce'].sample(n=4770, random_state=42)

# ---------------- Combine all samples ----------------

combined_df = pd.concat([
    bot_df, benign_df,
    infiltration_df,
    brute_web_df, brute_xss_df, sql_injection_df,
    ddos_hoic_df, ddos_loicudp_df, ddos_loichttp_df,
    dos_hulk_df, dos_slowhttp_df,
    dos_goldeneye_df, dos_slowloris_df,
    ftp_bruteforce_df, ssh_bruteforce_df
], ignore_index=True)

# ---------------- Save output ----------------

output_path = "/kaggle/working/combined_intrusion_dataset.csv"
combined_df.to_csv(output_path, index=False)

# ---------------- Print stats ----------------

print("Saved file:", output_path)
print("Final shape:", combined_df.shape)
print("\nLabel counts:\n", combined_df['Label'].value_counts())


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# 1️⃣ Load dataset
df = pd.read_csv("/kaggle/working/combined_intrusion_dataset.csv")

# 2️⃣ Remove columns where all values are NaN
df = df.dropna(axis=1, how='all')

# 3️⃣ Remove columns where all values are 0
df = df.loc[:, (df != 0).any(axis=0)]

# 4️⃣ Remove duplicate rows
df = df.drop_duplicates()

# 5️⃣ Identify column types
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
cat_cols = df.select_dtypes(include=['object']).columns

# 6️⃣ Handle missing values
df[num_cols] = df[num_cols].fillna(df[num_cols].median())
df[cat_cols] = df[cat_cols].fillna(df[cat_cols].mode().iloc[0])

# 7️⃣ Label encode categorical columns
le = LabelEncoder()
for col in cat_cols:
    df[col] = le.fit_transform(df[col].astype(str))

# 8️⃣ Min-Max normalization
scaler = MinMaxScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

# 9️⃣ Save final output in Kaggle working directory
output_filename = "/kaggle/working/ids2018_cleaned_combined.csv"
df.to_csv(output_filename, index=False)

print("✅ Preprocessing complete!")
print("📁 Saved as:", output_filename)
print("Shape:", df.shape)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# ---------- 1. Load (avoid low-memory dtype guessing) ----------
input_path = "/kaggle/working/combined_intrusion_dataset.csv"
df = pd.read_csv(input_path, low_memory=False)   # prevents chunked dtype inference warning

# Quick info to debug if needed
print("Initial shape:", df.shape)
print("Initial dtypes (sample):\n", df.dtypes.head(20))

# ---------- 2. Basic cleaning ----------
# Drop columns that are completely NaN
df = df.dropna(axis=1, how="all")

# Drop columns that are all zeros (if truly all zeros)
df = df.loc[:, (df != 0).any(axis=0)]

# Drop exact duplicate rows
df = df.drop_duplicates().reset_index(drop=True)

# ---------- 3. Detect candidate numeric columns robustly ----------
# We'll try to coerce each column to numeric and keep columns that convert well.
numeric_candidates = []
conversion_stats = {}
for col in df.columns:
    # try converting to numeric (coerce errors -> NaN)
    coerced = pd.to_numeric(df[col], errors="coerce")
    non_na_ratio = coerced.notna().sum() / len(coerced)
    conversion_stats[col] = non_na_ratio
    # if at least 80% of values convert to numeric, treat as numeric (you can adjust threshold)
    if non_na_ratio >= 0.80:
        numeric_candidates.append(col)

print(f"Detected {len(numeric_candidates)} numeric-like columns (>=80% convertible).")

# Force convert those columns to numeric (float64). Others kept as-is (likely categorical/text)
for col in numeric_candidates:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# ---------- 4. Inspect for infinities or extreme values ----------
# Columns that contain ±inf
inf_cols = []
for col in numeric_candidates:
    ser = df[col]
    if np.isinf(ser.to_numpy()).any():
        inf_cols.append(col)

print("Columns with ±inf values:", inf_cols)

# Replace ±inf with NaN so we can handle them with median filling
if inf_cols:
    df[numeric_candidates] = df[numeric_candidates].replace([np.inf, -np.inf], np.nan)

# Check for huge values that might overflow float64 when operations applied
# We'll compute absolute max and print columns with extremely large values (> 1e300)
huge_cols = []
for col in numeric_candidates:
    try:
        max_abs = np.nanmax(np.abs(df[col].to_numpy()))
        if np.isfinite(max_abs) and max_abs > 1e300:
            huge_cols.append((col, max_abs))
    except Exception:
        # if conversion to numpy fails, skip
        pass

print("Columns with extremely large magnitudes (>1e300):", huge_cols)

# If you see columns in huge_cols you might want to inspect or clip them.
# For safety we'll clip numeric values to a sane range before scaling (optional)
CLIP_LIMIT = 1e300
df[numeric_candidates] = df[numeric_candidates].apply(lambda s: s.clip(lower=-CLIP_LIMIT, upper=CLIP_LIMIT))

# ---------- 5. Recompute numeric / categorical column lists ----------
num_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
print("Final numeric columns count:", len(num_cols))
print("Final categorical columns count:", len(cat_cols))

# ---------- 6. Handle missing values ----------
# Numeric: fill with median
if len(num_cols) > 0:
    df[num_cols] = df[num_cols].fillna(df[num_cols].median())

# Categorical: fill with mode (most frequent)
for col in cat_cols:
    if df[col].isna().any():
        mode_val = df[col].mode(dropna=True)
        if len(mode_val) > 0:
            df[col] = df[col].fillna(mode_val.iloc[0])
        else:
            # fallback to empty string if mode can't be computed
            df[col] = df[col].fillna("")

# ---------- 7. Encode categorical columns ----------
le = LabelEncoder()
for col in cat_cols:
    # convert to string first to avoid issues with mixed types
    df[col] = le.fit_transform(df[col].astype(str))

# ---------- 8. Final check for non-finite numerics before scaling ----------
# Replace any remaining inf/-inf with NaN (just in case), then fill with medians again
df[num_cols] = df[num_cols].replace([np.inf, -np.inf], np.nan)
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

# Ensure all numeric values are finite now
finite_check = {col: np.isfinite(df[col].to_numpy()).all() for col in num_cols}
bad = [c for c, ok in finite_check.items() if not ok]
print("Columns still non-finite (should be empty):", bad)

# ---------- 9. Min-Max scaling ----------
scaler = MinMaxScaler()
if len(num_cols) > 0:
    df[num_cols] = scaler.fit_transform(df[num_cols])

# ---------- 10. Save and report ----------
output_filename = "/kaggle/working/ids2018_cleaned_combined.csv"
df.to_csv(output_filename, index=False)

print("✅ Preprocessing complete!")
print("📁 Saved as:", output_filename)
print("Shape:", df.shape)
print("Label distribution (if present):")
if 'Label' in df.columns:
    print(df['Label'].value_counts())
else:
    print("No 'Label' column found in final dataframe.")


In [65]:
print("hi")

hi


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from catboost import CatBoostClassifier
import time

# -----------------------------------------
# 1️⃣ Load cleaned dataset
# -----------------------------------------
df = pd.read_csv("/kaggle/working/ids2018_cleaned_combined_1.csv")

# Ensure "Label" column exists
if "Label" not in df.columns:
    raise ValueError("❌ ERROR: 'Label' column not found in dataset!")

# -----------------------------------------
# 2️⃣ Split features and target
# -----------------------------------------
X = df.drop("Label", axis=1)
y = df["Label"]

# -----------------------------------------
# 3️⃣ Train-test split (80-20)
# -----------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

# -----------------------------------------
# 4️⃣ Train CatBoost model + record time
# -----------------------------------------
model = CatBoostClassifier(
    iterations=300,
    learning_rate=0.1,
    depth=8,
    loss_function='MultiClass',
    verbose=False
)

start_time = time.time()
model.fit(X_train, y_train)
end_time = time.time()

training_time = end_time - start_time

# -----------------------------------------
# 5️⃣ Predictions
# -----------------------------------------
y_pred = model.predict(X_test)
y_pred = y_pred.flatten()   # CatBoost outputs shape (n,1)

# -----------------------------------------
# 6️⃣ Metrics
# -----------------------------------------
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average="weighted")
recall = recall_score(y_test, y_pred, average="weighted")
precision = precision_score(y_test, y_pred, average="weighted")

# -----------------------------------------
# 7️⃣ Print results
# -----------------------------------------
print("✔️ CatBoost Training Complete\n")
print("📌 Accuracy:", acc)
print("📌 F1 Score (weighted):", f1)
print("📌 Recall (weighted):", recall)
print("📌 Precision (weighted):", precision)
print(f"⏱️ Time Taken to Train: {training_time:.2f} seconds")


In [42]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from catboost import CatBoostClassifier
import time
import pickle

# -----------------------------------------
# 1️⃣ Load cleaned dataset
# -----------------------------------------
df = pd.read_csv("/kaggle/input/cleaned-ids/ids2018_cleaned_combined_1.csv")

# Ensure "Label" column exists
if "Label" not in df.columns:
    raise ValueError("❌ ERROR: 'Label' column not found in dataset!")

# -----------------------------------------
# 2️⃣ Split features and target
# -----------------------------------------
X = df.drop("Label", axis=1)
y = df["Label"]

# -----------------------------------------
# 3️⃣ Train-test split (80-20)
# -----------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

# -----------------------------------------
# 4️⃣ Train CatBoost model + record time
# -----------------------------------------
model = CatBoostClassifier(
    iterations=300,
    learning_rate=0.1,
    depth=8,
    loss_function='MultiClass',
    verbose=False
)

start_time = time.time()
model.fit(X_train, y_train)
end_time = time.time()
training_time = end_time - start_time

# -----------------------------------------
# 5️⃣ Predictions
# -----------------------------------------
y_pred = model.predict(X_test)
y_pred = y_pred.flatten()

# -----------------------------------------
# 6️⃣ Metrics
# -----------------------------------------
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average="weighted")
recall = recall_score(y_test, y_pred, average="weighted")
precision = precision_score(y_test, y_pred, average="weighted")

# -----------------------------------------
# 7️⃣ Save model to Pickle
# -----------------------------------------
pickle_path = "/kaggle/working/catboost_ids2018_model.pkl"
with open(pickle_path, "wb") as f:
    pickle.dump(model, f)

# -----------------------------------------
# 8️⃣ Print results
# -----------------------------------------
print("✔️ CatBoost Training Complete\n")
print("📁 Model saved as:", pickle_path)
print("📌 Accuracy:", acc)
print("📌 F1 Score (weighted):", f1)
print("📌 Recall (weighted):", recall)
print("📌 Precision (weighted):", precision)
print(f"⏱️ Time Taken to Train: {training_time:.2f} seconds")


✔️ CatBoost Training Complete

📁 Model saved as: /kaggle/working/catboost_ids2018_model.pkl
📌 Accuracy: 0.999130923776903
📌 F1 Score (weighted): 0.99913091257803
📌 Recall (weighted): 0.999130923776903
📌 Precision (weighted): 0.9991315336373293
⏱️ Time Taken to Train: 22.72 seconds


above asnwer is after cleaning before feature selection

In [66]:
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from catboost import CatBoostClassifier

# Load
df = pd.read_csv("/kaggle/input/before-clean-ids/combined_intrusion_dataset.csv", low_memory=False)
df['Label'] = df['Label'].astype(int)

# X / y
X = df.drop(columns=['Label']).copy()
y = df['Label']

# Detect categorical columns (object dtype)
cat_cols = X.select_dtypes(include=['object']).columns.tolist()
print("Categorical columns detected (from X):", cat_cols)

# Replace NaN in categorical columns with a token and cast to string
for c in cat_cols:
    X[c] = X[c].fillna("__missing__").astype(str)

# Now compute cat feature indices relative to X
cat_feature_indices = [X.columns.get_loc(c) for c in cat_cols]
print("Cat feature indices:", cat_feature_indices)

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

# Train CatBoost (binary)
model = CatBoostClassifier(
    iterations=300,
    learning_rate=0.1,
    depth=8,
    loss_function='Logloss',
    verbose=False
)

start = time.time()
model.fit(X_train, y_train, cat_features=cat_feature_indices)
train_time = time.time() - start

# Predict & metrics
y_pred = model.predict(X_test).astype(int).flatten()
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='binary', zero_division=0)
recall = recall_score(y_test, y_pred, average='binary', zero_division=0)
f1 = f1_score(y_test, y_pred, average='binary', zero_division=0)

print("\n✔ RAW CatBoost (Option A) done")
print(f"⏱ Training time: {train_time:.2f} s")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")


ValueError: invalid literal for int() with base 10: 'Bot'

In [67]:
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from catboost import CatBoostClassifier

# -----------------------------------------------------
# 1) LOAD DATA
# -----------------------------------------------------
df = pd.read_csv("/kaggle/input/before-clean-ids/combined_intrusion_dataset.csv", low_memory=False)
print("Loaded dataset:", df.shape)

# -----------------------------------------------------
# 2) CONVERT LABEL COLUMN
#    Benign → 0
#    All other attack labels → 1
# -----------------------------------------------------
df['Label'] = df['Label'].astype(str).str.strip().str.lower()

df['Label'] = df['Label'].apply(
    lambda x: 0 if x in ("benign", "normal") or "benign" in x else 1
)

df['Label'] = df['Label'].astype(int)
print("Label distribution:\n", df['Label'].value_counts())

# -----------------------------------------------------
# 3) SEPARATE FEATURES + TARGET
# -----------------------------------------------------
X = df.drop(columns=['Label']).copy()
y = df['Label']

# Detect categorical (object/string) columns
cat_cols = X.select_dtypes(include=['object']).columns.tolist()
print("\nCategorical columns detected:", cat_cols)

# Fill NaN in categorical columns and convert to string
for col in cat_cols:
    X[col] = X[col].fillna("__missing__").astype(str)

# Convert categorical column names → column indices for CatBoost
cat_feature_indices = [X.columns.get_loc(c) for c in cat_cols]
print("Categorical feature indices:", cat_feature_indices)

# -----------------------------------------------------
# 4) TRAIN/TEST SPLIT
# -----------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

# -----------------------------------------------------
# 5) TRAIN CATBOOST CLASSIFIER
# -----------------------------------------------------
model = CatBoostClassifier(
    iterations=300,
    learning_rate=0.1,
    depth=8,
    loss_function='Logloss',
    verbose=False
)

start = time.time()
model.fit(X_train, y_train, cat_features=cat_feature_indices)
train_time = time.time() - start

# -----------------------------------------------------
# 6) PREDICT + METRICS
# -----------------------------------------------------
y_pred = model.predict(X_test).astype(int).flatten()

accuracy  = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=0)
recall    = recall_score(y_test, y_pred, zero_division=0)
f1        = f1_score(y_test, y_pred, zero_division=0)

# -----------------------------------------------------
# 7) PRINT RESULTS
# -----------------------------------------------------
print("\n✔ CatBoost Training Complete")
print(f"⏱ Training time: {train_time:.2f} seconds")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")


Loaded dataset: (100000, 84)
Label distribution:
 Label
1    50000
0    50000
Name: count, dtype: int64

Categorical columns detected: ['Timestamp', 'Flow ID', 'Src IP', 'Dst IP']
Categorical feature indices: [2, 79, 80, 82]

✔ CatBoost Training Complete
⏱ Training time: 32.93 seconds
Accuracy : 0.9853
Precision: 0.9972
Recall   : 0.9734
F1 Score : 0.9852


above is before cleaning

In [ ]:
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from catboost import CatBoostClassifier

# Load
df = pd.read_csv("/kaggle/working/combined_intrusion_dataset_binary.csv", low_memory=False)
df['Label'] = df['Label'].astype(int)

# Drop IP columns to reduce memory / speed impact
drop_cols = ['Src IP', 'Dst IP']  # change if names differ
df = df.drop(columns=[c for c in drop_cols if c in df.columns])

# X / y
X = df.drop(columns=['Label']).copy()
y = df['Label']

# Detect categorical columns (object dtype)
cat_cols = X.select_dtypes(include=['object']).columns.tolist()
print("Categorical columns detected (from X):", cat_cols)

# Replace NaN in categorical columns with a token and cast to string
for c in cat_cols:
    X[c] = X[c].fillna("__missing__").astype(str)

cat_feature_indices = [X.columns.get_loc(c) for c in cat_cols]
print("Cat feature indices:", cat_feature_indices)

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

# Train CatBoost (binary)
model = CatBoostClassifier(
    iterations=300,
    learning_rate=0.1,
    depth=8,
    loss_function='Logloss',
    verbose=False
)

start = time.time()
model.fit(X_train, y_train, cat_features=cat_feature_indices)
train_time = time.time() - start

# Predict & metrics
y_pred = model.predict(X_test).astype(int).flatten()
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='binary', zero_division=0)
recall = recall_score(y_test, y_pred, average='binary', zero_division=0)
f1 = f1_score(y_test, y_pred, average='binary', zero_division=0)

print("\n✔ RAW CatBoost (Option B) done")
print(f"⏱ Training time: {train_time:.2f} s")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")


above model is before cleaning dropping src and dest ip

In [ ]:
import pandas as pd

cleaned = pd.read_csv("/kaggle/working/ids2018_cleaned_combined_1.csv", low_memory=False)
print("dtype of Label:", cleaned['Label'].dtype)
print("Unique label values (sample):", cleaned['Label'].unique()[:30])
print("\nValue counts:\n", cleaned['Label'].value_counts().head(30))


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# ---------- 1. Load (avoid low-memory dtype guessing) ----------
input_path = "/kaggle/working/combined_intrusion_dataset.csv"
df = pd.read_csv(input_path, low_memory=False)

print("Initial shape:", df.shape)

# ---------- 2. Convert Label → Binary BEFORE cleaning ----------
# Benign = 0
# Any attack = 1
df['Label'] = (df['Label'] != 'Benign').astype(int)

print("Binary Label distribution:\n", df['Label'].value_counts())

# ---------- 3. Basic cleaning ----------
df = df.dropna(axis=1, how="all")          # drop all-NaN columns
df = df.loc[:, (df != 0).any(axis=0)]      # drop all-zero columns
df = df.drop_duplicates().reset_index(drop=True)

# ---------- 4. Detect numeric-like columns ----------
numeric_candidates = []
for col in df.columns:
    coerced = pd.to_numeric(df[col], errors="coerce")
    ratio = coerced.notna().sum() / len(coerced)
    if ratio >= 0.80:
        numeric_candidates.append(col)
        df[col] = coerced

# Ensure Label is treated as numeric (keep it as binary!)
if 'Label' not in numeric_candidates:
    numeric_candidates.append('Label')

# ---------- 5. Fix infinities ----------
df[numeric_candidates] = df[numeric_candidates].replace([np.inf, -np.inf], np.nan)

# ---------- 6. Clip extremely large values ----------
CLIP_LIMIT = 1e300
df[numeric_candidates] = df[numeric_candidates].apply(lambda s: s.clip(-CLIP_LIMIT, CLIP_LIMIT))

# ---------- 7. Identify final numeric + categorical ----------
num_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Remove LABEL so it never gets encoded
cat_cols = df.select_dtypes(include=['object']).columns.tolist()

print("Numeric columns:", len(num_cols))
print("Categorical columns:", len(cat_cols))

# ---------- 8. Handle missing values ----------
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

for col in cat_cols:
    df[col] = df[col].fillna(df[col].mode(dropna=True).iloc[0])

# ---------- 9. Encode categorical columns (not Label) ----------
le = LabelEncoder()
for col in cat_cols:
    df[col] = le.fit_transform(df[col].astype(str))

# ---------- 10. Final check before scaling ----------
df[num_cols] = df[num_cols].replace([np.inf, -np.inf], np.nan)
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

# ---------- 11. Scale numeric features (Label excluded automatically) ----------
scaler = MinMaxScaler()
cols_to_scale = [c for c in num_cols if c != 'Label']   # do NOT scale target

df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])

# ---------- 12. Save and report ----------
output_filename = "/kaggle/working/ids2018_cleaned_combined_1.csv"
df.to_csv(output_filename, index=False)

print("\n✅ Preprocessing complete!")
print("📁 Saved as:", output_filename)
print("Final shape:", df.shape)
print("Final Label distribution:\n", df['Label'].value_counts())


In [ ]:
import pandas as pd

# Load dataset
df = pd.read_csv("/kaggle/working/combined_intrusion_dataset.csv", low_memory=False)

# Convert Label -> binary
df['Label'] = (df['Label'] != 'Benign').astype(int)

# Save the output
output_path = "/kaggle/working/combined_intrusion_dataset_binary.csv"
df.to_csv(output_path, index=False)

print("✔ Binary conversion complete!")
print("📁 Saved as:", output_path)
print(df['Label'].value_counts())


In [ ]:
# hybrid_hlo_union_only.py
# Reduced-budget hybrid pipeline: PSO, GA, GWO -> UNION -> HLO -> Hill-climb -> final CatBoost (save)
# Prints selected features after PSO/GA/GWO and the final union members.
# Runs on Kaggle input path by default and prints final test metrics (accuracy, precision, recall, f1),
# confusion matrix and classification report.

import time
import pickle
import numpy as np
import pandas as pd
import warnings
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer, classification_report, confusion_matrix
from sklearn.base import clone

warnings.filterwarnings("ignore")
np.random.seed(42)

# -------------------- USER / EXPERIMENT SETTINGS --------------------
# Kaggle path requested by you:
CSV_PATH = "/kaggle/working/ids2018_cleaned_combined_1.csv"

TARGET_COL = "Label"   # change if your dataset uses another column name
MODEL_VERBOSE = 0            # CatBoost verbosity: 0 = silent
RANDOM_STATE = 42

# ---------- Reduced budgets (set to 20 for the three optimizers as requested) ----------
PSO_SWARM = 15
PSO_ITERS = 5     # <<-- set to 20

GA_POP = 30
GA_GENS = 5       # <<-- set to 20

GWO_WOLVES = 10
GWO_ITERS = 5     # <<-- set to 20

HLO_POP = 15
HLO_ITERS = 8
HLO_TEACHER_FACTOR = 0.75
HLO_MUTATION = 0.12

# Greedy hill-climb
HILLCLIMB_MAX_STEPS = 100
HILLCLIMB_EVAL_CAP = 500

# CV folds
CV_OPT = 2
CV_FINAL = 5

# CatBoost iterations
CB_ITER_OPT = 100
CB_ITER_HLO = 200
CB_ITER_FINAL = 500

FINAL_TEST_SIZE = 0.2
SAVE_PREFIX = "hybrid_hlo_union"
# ------------------------------------------------------------------------


# -------------------- Load data (robust handling of messy column names) --------------------
print(f"[{time.strftime('%H:%M:%S')}] Loading CSV from: {CSV_PATH}")
df = pd.read_csv(CSV_PATH, low_memory=False)
print(f"[{time.strftime('%H:%M:%S')}] Raw loaded shape: {df.shape}")
print(f"[{time.strftime('%H:%M:%S')}] Raw columns sample: {df.columns.tolist()[:12]}")

# Clean column names: strip whitespace and normalize repeated spaces
df.columns = df.columns.astype(str).str.strip().str.replace(r"\s+", " ", regex=True)
print(f"[{time.strftime('%H:%M:%S')}] Cleaned columns sample: {df.columns.tolist()[:12]}")

# If an index column like 'Unnamed: 0' exists (common from CSV exports), drop it
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])
    print(f"[{time.strftime('%H:%M:%S')}] Dropped 'Unnamed: 0' column. New shape: {df.shape}")

# If the requested TARGET_COL isn't found, try to auto-detect a label-like column (case-insensitive)
if TARGET_COL not in df.columns:
    # try case-insensitive match
    cols_lower = {c.lower(): c for c in df.columns}
    if TARGET_COL.lower() in cols_lower:
        real_col = cols_lower[TARGET_COL.lower()]
        print(f"[{time.strftime('%H:%M:%S')}] Using case-insensitive match for target: '{real_col}'")
        TARGET_COL = real_col
    else:
        # fallback: search for any column name that contains 'label' or 'target'
        cand = [c for c in df.columns if 'label' in c.lower() or 'target' in c.lower()]
        if len(cand) == 1:
            print(f"[{time.strftime('%H:%M:%S')}] Auto-detected target column: '{cand[0]}'")
            TARGET_COL = cand[0]
        elif len(cand) > 1:
            print(f"[{time.strftime('%H:%M:%S')}] Multiple candidate target columns found: {cand}. Using first: '{cand[0]}'")
            TARGET_COL = cand[0]
        else:
            raise ValueError(f"Target column '{TARGET_COL}' not found (after cleaning). Columns: {df.columns.tolist()[:12]}...")

print(f"[{time.strftime('%H:%M:%S')}] Using TARGET_COL = '{TARGET_COL}'")

# Basic preprocessing expectation: ensure no object columns remain unencoded for CatBoost.
from sklearn.preprocessing import LabelEncoder
obj_cols = df.select_dtypes(include=["object"]).columns.tolist()
if obj_cols:
    print(f"[{time.strftime('%H:%M:%S')}] Label-encoding object columns for safe use: {obj_cols}")
    for c in obj_cols:
        df[c] = df[c].astype(str).fillna("NA")
        df[c] = LabelEncoder().fit_transform(df[c])

# Ensure no NaNs in features/target used by optimizers
df = df.dropna(axis=0).reset_index(drop=True)

# Prepare X, y
X = df.drop(TARGET_COL, axis=1)
y = df[TARGET_COL].astype(int)
FEATURE_NAMES = X.columns.tolist()
N_FEATURES = X.shape[1]
print(f"[{time.strftime('%H:%M:%S')}] Prepared X ({X.shape}) and y ({y.shape}). Number of features: {N_FEATURES}")


# -------------------- CatBoost factory --------------------
def get_catboost_model(iterations=100):
    try:
        from catboost import CatBoostClassifier
    except Exception as e:
        raise ImportError("catboost not installed. Install with: pip install catboost") from e
    return CatBoostClassifier(iterations=iterations, learning_rate=0.05, depth=6,
                              verbose=MODEL_VERBOSE, random_seed=RANDOM_STATE, thread_count=-1)

# -------------------- Fitness cache --------------------
fitness_cache = {}
def key_from_mask(mask_bool):
    return tuple(sorted(np.where(np.array(mask_bool).astype(bool))[0].tolist()))

def evaluate_mask_global(mask_bool, cv=CV_OPT, cb_iter=CB_ITER_OPT):
    key = key_from_mask(mask_bool)
    if key in fitness_cache:
        return fitness_cache[key]
    if len(key) == 0:
        fitness_cache[key] = 0.0
        return 0.0

    X_sel = X.iloc[:, list(key)]
    model = get_catboost_model(iterations=cb_iter)
    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=RANDOM_STATE)

    try:
        accs = cross_val_score(clone(model), X_sel, y, cv=skf, scoring="accuracy", n_jobs=-1)
        precs = cross_val_score(clone(model), X_sel, y, cv=skf, scoring=make_scorer(precision_score, zero_division=0), n_jobs=-1)
        recs = cross_val_score(clone(model), X_sel, y, cv=skf, scoring=make_scorer(recall_score, zero_division=0), n_jobs=-1)
        f1s = cross_val_score(clone(model), X_sel, y, cv=skf, scoring=make_scorer(f1_score, zero_division=0), n_jobs=-1)
        score = float((np.mean(accs) + np.mean(precs) + np.mean(recs) + np.mean(f1s)) / 4.0)
    except Exception as e:
        # if a training error occurs (e.g., degenerate feature set), return 0
        score = 0.0

    fitness_cache[key] = score
    return score

# -------------------- Helpers --------------------
def mask_to_features(mask):
    idxs = np.where(np.array(mask).astype(bool))[0].tolist()
    return [FEATURE_NAMES[i] for i in idxs]

def log(msg):
    print(f"[{time.strftime('%H:%M:%S')}] {msg}", flush=True)

# -------------------- PSO (binary) --------------------
def run_pso(swarm_size=PSO_SWARM, iters=PSO_ITERS, cv=CV_OPT):
    log(f"PSO START (swarm={swarm_size}, iters={iters}, cv={cv})")
    t0 = time.time()
    dim = N_FEATURES
    pos = np.random.randint(0,2,(swarm_size,dim)).astype(int)
    vel = np.random.uniform(-1,1,(swarm_size,dim))

    pbest = pos.copy()
    pbest_scores = np.array([evaluate_mask_global(p.astype(bool), cv=cv, cb_iter=CB_ITER_OPT) for p in pos])

    gbest_idx = int(np.argmax(pbest_scores))
    gbest = pbest[gbest_idx].copy()
    gbest_score = pbest_scores[gbest_idx]

    w = 0.6; c1 = c2 = 1.5
    for t in range(iters):
        log(f" PSO iter {t+1}/{iters} best_global={gbest_score:.4f}")
        for i in range(swarm_size):
            r1 = np.random.rand(dim); r2 = np.random.rand(dim)
            vel[i] = w*vel[i] + c1*r1*(pbest[i] - pos[i]) + c2*r2*(gbest - pos[i])
            s = 1.0 / (1.0 + np.exp(-vel[i]))
            pos[i] = (np.random.rand(dim) < s).astype(int)

            sc = evaluate_mask_global(pos[i].astype(bool), cv=cv, cb_iter=CB_ITER_OPT)
            if sc > pbest_scores[i]:
                pbest[i] = pos[i].copy()
                pbest_scores[i] = sc
            if sc > gbest_score:
                gbest = pos[i].copy()
                gbest_score = sc
        w = max(0.2, w*0.97)

    best_idx = int(np.argmax(pbest_scores))
    best_mask = pbest[best_idx].copy()
    best_score = pbest_scores[best_idx]
    t1 = time.time()
    log(f"PSO DONE in {int(t1-t0)}s best_score={best_score:.4f} selected={int(np.sum(best_mask))}")
    return best_mask, best_score, int(t1-t0)

# -------------------- GA (binary) --------------------
def run_ga(pop_size=GA_POP, gens=GA_GENS, cv=CV_OPT):
    log(f"GA START (pop={pop_size}, gens={gens}, cv={cv})")
    t0 = time.time()
    dim = N_FEATURES
    pop = np.random.randint(0,2,(pop_size, dim)).astype(int)
    fitness_scores = np.array([evaluate_mask_global(ind.astype(bool), cv=cv, cb_iter=CB_ITER_OPT) for ind in pop])

    def tournament_select(k=3):
        idxs = np.random.randint(0, pop_size, k)
        return idxs[np.argmax(fitness_scores[idxs])]

    for g in range(gens):
        log(f" GA gen {g+1}/{gens} current_best={np.max(fitness_scores):.4f}")
        new_pop = []
        # elitism
        elite_idxs = np.argsort(fitness_scores)[-2:]
        new_pop.extend(pop[elite_idxs].tolist())

        while len(new_pop) < pop_size:
            i1 = tournament_select(); i2 = tournament_select()
            p1 = pop[i1].copy(); p2 = pop[i2].copy()
            # crossover
            if np.random.rand() < 0.7:
                pt = np.random.randint(1, dim)
                c1 = np.concatenate([p1[:pt], p2[pt:]])
                c2 = np.concatenate([p2[:pt], p1[pt:]])
            else:
                c1, c2 = p1, p2
            # mutation
            for child in (c1, c2):
                for d in range(dim):
                    if np.random.rand() < 0.1:
                        child[d] = 1 - child[d]
                new_pop.append(child)
                if len(new_pop) >= pop_size:
                    break
        pop = np.array(new_pop[:pop_size])
        fitness_scores = np.array([evaluate_mask_global(ind.astype(bool), cv=cv, cb_iter=CB_ITER_OPT) for ind in pop])

    best_idx = int(np.argmax(fitness_scores))
    best_mask = pop[best_idx].copy()
    best_score = fitness_scores[best_idx]
    t1 = time.time()
    log(f"GA DONE in {int(t1-t0)}s best_score={best_score:.4f} selected={int(np.sum(best_mask))}")
    return best_mask, best_score, int(t1-t0)

# -------------------- GWO (binary) --------------------
def run_gwo(wolves=GWO_WOLVES, iters=GWO_ITERS, cv=CV_OPT):
    log(f"GWO START (wolves={wolves}, iters={iters}, cv={cv})")
    t0 = time.time()
    dim = N_FEATURES
    pop = np.random.randint(0,2,(wolves, dim)).astype(int)
    fitness_scores = np.array([evaluate_mask_global(ind.astype(bool), cv=cv, cb_iter=CB_ITER_OPT) for ind in pop])

    Alpha = Beta = Delta = None
    Alpha_score = Beta_score = Delta_score = -1.0

    for itr in range(iters):
        log(f" GWO iter {itr+1}/{iters} best_alpha={Alpha_score:.4f}")
        for i in range(wolves):
            sc = fitness_scores[i]
            if sc > Alpha_score:
                Delta_score, Beta_score, Alpha_score = Beta_score, Alpha_score, sc
                Delta, Beta, Alpha = Beta, Alpha, pop[i].copy()
            elif sc > Beta_score:
                Delta_score, Beta_score = Beta_score, sc
                Delta, Beta = Beta, pop[i].copy()
            elif sc > Delta_score:
                Delta_score = sc
                Delta = pop[i].copy()

        a = 2 - itr * (2.0 / iters)
        for i in range(wolves):
            for d in range(dim):
                if Alpha is None:
                    continue
                r1, r2 = np.random.rand(), np.random.rand()
                A1 = 2 * a * r1 - a; C1 = 2 * r2
                D_alpha = abs(C1 * Alpha[d] - pop[i][d])
                X1 = Alpha[d] - A1 * D_alpha

                r1, r2 = np.random.rand(), np.random.rand()
                A2 = 2 * a * r1 - a; C2 = 2 * r2
                D_beta = abs(C2 * Beta[d] - pop[i][d])
                X2 = Beta[d] - A2 * D_beta

                r1, r2 = np.random.rand(), np.random.rand()
                A3 = 2 * a * r1 - a; C3 = 2 * r2
                D_delta = abs(C3 * Delta[d] - pop[i][d])
                X3 = Delta[d] - A3 * D_delta

                new_pos = (X1 + X2 + X3) / 3.0
                s = 1.0 / (1.0 + np.exp(-new_pos))
                pop[i][d] = 1 if np.random.rand() < s else 0

        fitness_scores = np.array([evaluate_mask_global(ind.astype(bool), cv=cv, cb_iter=CB_ITER_OPT) for ind in pop])

    best_idx = int(np.argmax(fitness_scores))
    best_mask = pop[best_idx].copy()
    best_score = fitness_scores[best_idx]
    t1 = time.time()
    log(f"GWO DONE in {int(t1-t0)}s best_score={best_score:.4f} selected={int(np.sum(best_mask))}")
    return best_mask, best_score, int(t1-t0)

# -------------------- UNION (only) --------------------
def get_union_mask(*masks):
    union_idx = set()
    for m in masks:
        idxs = np.where(np.array(m).astype(bool))[0].tolist()
        union_idx.update(idxs)
    mask = np.zeros(N_FEATURES, dtype=int)
    for i in union_idx:
        mask[i] = 1
    return mask

# -------------------- HLO on candidates --------------------
def hlo_on_candidates(candidate_mask, pop_size=HLO_POP, iters=HLO_ITERS, cv=CV_OPT):
    candidate_indices = np.where(np.array(candidate_mask).astype(bool))[0].tolist()
    k = len(candidate_indices)
    if k == 0:
        raise ValueError("Candidate set is empty.")

    log(f"HLO START on {k} candidate features (pop={pop_size}, iters={iters})")
    t0 = time.time()

    pop = np.random.randint(0,2,(pop_size, k)).astype(int)

    def fitness_candidate(bitmask):
        full_mask = np.zeros(N_FEATURES, dtype=int)
        for j,bit in enumerate(bitmask):
            if bit == 1:
                full_mask[candidate_indices[j]] = 1
        return evaluate_mask_global(full_mask.astype(bool), cv=cv, cb_iter=CB_ITER_HLO)

    fitness_scores = np.array([fitness_candidate(ind) for ind in pop])
    best_idx = int(np.argmax(fitness_scores))
    best_solution = pop[best_idx].copy()
    best_score = fitness_scores[best_idx]

    for it in range(iters):
        log(f" HLO iter {it+1}/{iters} current_best={best_score:.4f}")
        teacher = pop[int(np.argmax(fitness_scores))].copy()
        new_pop = []
        for i in range(pop_size):
            learner = pop[i].copy()
            # teaching phase
            for d in range(k):
                if np.random.rand() < HLO_TEACHER_FACTOR:
                    learner[d] = teacher[d]
            # peer learning
            partner = pop[np.random.randint(pop_size)].copy()
            for d in range(k):
                if learner[d] != partner[d] and np.random.rand() < 0.5:
                    learner[d] = partner[d]
            # mutation
            for d in range(k):
                if np.random.rand() < HLO_MUTATION:
                    learner[d] = 1 - learner[d]
            new_pop.append(learner)
        pop = np.array(new_pop)
        fitness_scores = np.array([fitness_candidate(ind) for ind in pop])
        gen_best_idx = int(np.argmax(fitness_scores))
        gen_best_score = fitness_scores[gen_best_idx]
        gen_best_sol = pop[gen_best_idx].copy()
        if gen_best_score > best_score:
            best_score = gen_best_score
            best_solution = gen_best_sol.copy()

    # map back to full mask
    final_full_mask = np.zeros(N_FEATURES, dtype=int)
    for j,bit in enumerate(best_solution):
        if bit == 1:
            final_full_mask[candidate_indices[j]] = 1

    t1 = time.time()
    log(f"HLO DONE in {int(t1-t0)}s best_score={best_score:.4f} final_selected={int(np.sum(final_full_mask))}")
    return final_full_mask, best_score, int(t1-t0)

# -------------------- Greedy Hill-Climb (local search) --------------------
def hill_climb_on_candidates(initial_mask, candidate_mask, max_steps=HILLCLIMB_MAX_STEPS, eval_cap=HILLCLIMB_EVAL_CAP, cv=CV_OPT):
    candidate_indices = np.where(np.array(candidate_mask).astype(bool))[0].tolist()
    if len(candidate_indices) == 0:
        log("Hill-climb: candidate set empty, skipping.")
        return initial_mask, 0.0, 0

    log(f"Hill-climb START over {len(candidate_indices)} candidates (max_steps={max_steps}, eval_cap={eval_cap})")
    t0 = time.time()
    current_mask = initial_mask.copy()
    current_score = evaluate_mask_global(current_mask.astype(bool), cv=cv, cb_iter=CB_ITER_HLO)
    evals = 0
    steps = 0
    improved = True

    while improved and steps < max_steps and evals < eval_cap:
        improved = False
        for idx in np.random.permutation(candidate_indices):
            trial_mask = current_mask.copy()
            trial_mask[idx] = 1 - trial_mask[idx]  # flip
            trial_score = evaluate_mask_global(trial_mask.astype(bool), cv=cv, cb_iter=CB_ITER_HLO)
            evals += 1
            if trial_score > current_score + 1e-8:
                current_mask = trial_mask
                current_score = trial_score
                improved = True
                steps += 1
                log(f" Hill-climb step {steps}: flipped {FEATURE_NAMES[idx]} -> new_score={current_score:.4f} (evals={evals})")
                break
            if evals >= eval_cap or steps >= max_steps:
                break
    t1 = time.time()
    log(f"Hill-climb DONE in {int(t1-t0)}s steps={steps} evals={evals} final_score={current_score:.4f} selected={int(np.sum(current_mask))}")
    return current_mask, current_score, int(t1-t0)

# -------------------- Final evaluation (5-fold CV) --------------------
def final_evaluation(mask_bool, cv=CV_FINAL, cb_iter=CB_ITER_FINAL):
    idxs = np.where(np.array(mask_bool).astype(bool))[0].tolist()
    if len(idxs) == 0:
        raise ValueError("Final mask selects zero features.")
    X_sel = X.iloc[:, idxs]
    model = get_catboost_model(iterations=cb_iter)
    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=RANDOM_STATE)
    accs = []; precs = []; recs = []; f1s = []
    t0 = time.time()
    for tr,te in skf.split(X_sel, y):
        m = clone(model); m.fit(X_sel.iloc[tr], y.iloc[tr])
        pred = m.predict(X_sel.iloc[te])
        accs.append(accuracy_score(y.iloc[te], pred))
        precs.append(precision_score(y.iloc[te], pred, zero_division=0))
        recs.append(recall_score(y.iloc[te], pred, zero_division=0))
        f1s.append(f1_score(y.iloc[te], pred, zero_division=0))
    t1 = time.time()
    results = {
        "n_features": len(idxs),
        "features": [FEATURE_NAMES[i] for i in idxs],
        "acc_mean": float(np.mean(accs)), "acc_std": float(np.std(accs)),
        "prec_mean": float(np.mean(precs)), "prec_std": float(np.std(precs)),
        "rec_mean": float(np.mean(recs)), "rec_std": float(np.std(recs)),
        "f1_mean": float(np.mean(f1s)), "f1_std": float(np.std(f1s)),
        "eval_time_s": int(t1 - t0)
    }
    return results

# -------------------- MAIN PIPELINE --------------------
if __name__ == "__main__":
    total_t0 = time.time()
    log("===== HYBRID (reduced budget) + HLO + HILL-CLIMB (UNION only) START =====")

    # PSO
    pso_mask, pso_score, pso_time = run_pso(swarm_size=PSO_SWARM, iters=PSO_ITERS, cv=CV_OPT)
    pso_feats = mask_to_features(pso_mask)
    log(f"PSO selected ({len(pso_feats)}): {pso_feats}")

    # GA
    ga_mask, ga_score, ga_time = run_ga(pop_size=GA_POP, gens=GA_GENS, cv=CV_OPT)
    ga_feats = mask_to_features(ga_mask)
    log(f"GA selected ({len(ga_feats)}): {ga_feats}")

    # GWO
    gwo_mask, gwo_score, gwo_time = run_gwo(wolves=GWO_WOLVES, iters=GWO_ITERS, cv=CV_OPT)
    gwo_feats = mask_to_features(gwo_mask)
    log(f"GWO selected ({len(gwo_feats)}): {gwo_feats}")

    # Derive UNION of the three optimizers
    union_mask = get_union_mask(pso_mask, ga_mask, gwo_mask)
    union_feats = mask_to_features(union_mask)
    log(f"UNION candidate features ({len(union_feats)}): {union_feats}")

    # HLO on union
    if len(union_feats) == 0:
        log("UNION empty — nothing to optimize. Exiting.")
        raise SystemExit("No union features selected by optimizers.")

    hlo_mask, hlo_score, hlo_time = hlo_on_candidates(union_mask, pop_size=HLO_POP, iters=HLO_ITERS, cv=CV_OPT)
    hlo_feats = mask_to_features(hlo_mask)
    log(f"HLO final mask selected ({len(hlo_feats)}): {hlo_feats}")

    # Hill-climb restricted to union candidates
    hc_mask, hc_score, hc_time = hill_climb_on_candidates(hlo_mask, union_mask, max_steps=HILLCLIMB_MAX_STEPS, eval_cap=HILLCLIMB_EVAL_CAP, cv=CV_OPT)
    hc_feats = mask_to_features(hc_mask)
    log(f"Hill-climb final mask selected ({len(hc_feats)}): {hc_feats}")

    # Final CV evaluation (5-fold)
    final_res = final_evaluation(hc_mask, cv=CV_FINAL, cb_iter=CB_ITER_FINAL)
    log(f"Final CV (5-fold) | n_features={final_res['n_features']} | F1={final_res['f1_mean']:.4f} ± {final_res['f1_std']:.4f}")

    # Train final CatBoost on 80% and evaluate on 20%, save model
    selected_idxs = np.where(np.array(hc_mask).astype(bool))[0].tolist()
    selected_features = [FEATURE_NAMES[i] for i in selected_idxs]

    X_sel = X[selected_features]
    X_train, X_test, y_train, y_test = train_test_split(X_sel, y, test_size=FINAL_TEST_SIZE, stratify=y, random_state=RANDOM_STATE)

    model = get_catboost_model(iterations=CB_ITER_FINAL)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    test_acc = accuracy_score(y_test, y_pred)
    test_prec = precision_score(y_test, y_pred, zero_division=0)
    test_rec = recall_score(y_test, y_pred, zero_division=0)
    test_f1 = f1_score(y_test, y_pred, zero_division=0)
    test_cm = confusion_matrix(y_test, y_pred)
    test_report = classification_report(y_test, y_pred, zero_division=0)

    test_metrics = {
        'acc': float(test_acc), 'prec': float(test_prec), 'rec': float(test_rec), 'f1': float(test_f1),
        'n_test': int(X_test.shape[0]),
        'confusion_matrix': test_cm.tolist(),  # convert to list for pickle/json friendliness
        'classification_report': test_report
    }

    model_filename = f"{SAVE_PREFIX}_union_model.pkl"
    with open(model_filename, 'wb') as mf:
        pickle.dump(model, mf)

    log(f"Saved final CatBoost union model -> {model_filename} (test_f1={test_f1:.4f})")

    # Save aggregated results (only union)
    out = {
        "pso_mask": pso_mask, "pso_score": pso_score, "pso_time": pso_time,
        "ga_mask": ga_mask, "ga_score": ga_score, "ga_time": ga_time,
        "gwo_mask": gwo_mask, "gwo_score": gwo_score, "gwo_time": gwo_time,
        "union_mask": union_mask,
        "hlo_mask": hlo_mask, "hlo_score": hlo_score, "hlo_time": hlo_time,
        "hc_mask": hc_mask, "hc_score": hc_score, "hc_time": hc_time,
        "final_eval": final_res,
        "selected_features": selected_features,
        "model_file": model_filename,
        "test_metrics": test_metrics,
        "fitness_cache_len": len(fitness_cache)
    }
    with open(f"{SAVE_PREFIX}_results.pkl", "wb") as f:
        pickle.dump(out, f)

    total_t1 = time.time()
    elapsed_total = int(total_t1 - total_t0)
    log(f"PIPELINE COMPLETE in {elapsed_total}s. Results saved to {SAVE_PREFIX}_results.pkl and model {model_filename}")

    # Print short summary and explicit final test metrics (requested)
    print("\n=== SUMMARY ===")
    print(f"PSO selected ({len(pso_feats)}): {pso_feats}")
    print(f"GA selected  ({len(ga_feats)}): {ga_feats}")
    print(f"GWO selected ({len(gwo_feats)}): {gwo_feats}")
    print(f"UNION candidates ({len(union_feats)}): {union_feats}")
    print(f"HLO selected ({len(hlo_feats)}): {hlo_feats}")
    print(f"HILL-CLIMB selected ({len(hc_feats)}): {hc_feats}")
    print(f"Final CV F1: {final_res['f1_mean']:.4f} ± {final_res['f1_std']:.4f}")

    # Final test set metrics (explicit printout)
    print("\n--- FINAL TEST METRICS (80/20 held-out) ---")
    print(f"Test samples (n_test) : {test_metrics['n_test']}")
    print(f"Accuracy : {test_metrics['acc']:.4f}")
    print(f"Precision: {test_metrics['prec']:.4f}")
    print(f"Recall   : {test_metrics['rec']:.4f}")
    print(f"F1-score : {test_metrics['f1']:.4f}")
    print("\nConfusion Matrix (rows=true / cols=pred):")
    print(np.array(test_metrics['confusion_matrix']))
    print("\nClassification Report:")
    print(test_metrics['classification_report'])

    print(f"\nModel saved to: {model_filename}")

In [ ]:
# intersection_hlo_with_hillclimb_fast.py
# Pipeline (reduced budget + hill-climb) with UNION, INTERSECTION, and VOTING candidate flows:
#  PSO + GA + GWO (CatBoost fitness, lighter during opt) -> derive UNION / INTERSECTION / VOTING
#  For each candidate set: HLO (on candidates) -> Greedy hill-climb (restricted) -> Final CatBoost eval (5-fold CV)
#  Additionally: train a CatBoost model on 80% of the data and evaluate on the held-out 20% test set
#  Train & save a CatBoost model for each flow (union / intersection / voting) using the 80/20 split.
# Prints logs, mean ± std for metrics, stage timings, saves results and models.

import time
import pickle
import numpy as np
import pandas as pd
import warnings
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.base import clone

warnings.filterwarnings("ignore")
np.random.seed(42)

# -------------------- USER / EXPERIMENT SETTINGS --------------------
# If you prefer to load CSV instead, uncomment and change:
df = pd.read_csv("/kaggle/input/cleaned-ids/ids2018_cleaned_combined_1.csv")

TARGET_COL = "Label"   # target column
MODEL_VERBOSE = 0            # CatBoost verbosity: 0 = silent
RANDOM_STATE = 42

# ---------- Reduced budgets for faster runs (you can tune these) ----------
PSO_SWARM = 15   # reduced swarm
PSO_ITERS = 5   # reduced iterations

GA_POP = 30      # reduced population
GA_GENS = 5     # reduced generations

GWO_WOLVES = 10
GWO_ITERS = 5

HLO_POP = 15
HLO_ITERS = 5
HLO_TEACHER_FACTOR = 0.75
HLO_MUTATION = 0.12

# Greedy hill-climb after HLO
HILLCLIMB_MAX_STEPS = 100   # stop if no improvement or step limit
HILLCLIMB_EVAL_CAP = 500    # safety cap on evaluations (prevent runaway)

# CV folds
CV_OPT = 2    # cheaper CV during optimization + HLO (speed)
CV_FINAL = 5  # final evaluation (A1 requested)

# CatBoost iterations
CB_ITER_OPT = 100    # iterations during optimization (smaller)
CB_ITER_HLO = 200
CB_ITER_FINAL = 500  # final evaluation iterations (bigger)

# Train/test split for final saved models
FINAL_TEST_SIZE = 0.2

SAVE_PREFIX = "hybrid_hlo_models"
# ------------------------------------------------------------------------

# Ensure df exists
try:
    df
except NameError:
    raise RuntimeError("DataFrame `df` not found. Assign your dataset to variable `df` or load at top.")

# Prepare data
X = df.drop(TARGET_COL, axis=1)

y = df[TARGET_COL].astype(int)
FEATURE_NAMES = X.columns.tolist()
N_FEATURES = X.shape[1]

# -------------------- Model factory (CatBoost) --------------------
def get_catboost_model(iterations=100):
    try:
        from catboost import CatBoostClassifier
    except Exception as e:
        raise ImportError("catboost not installed. Install with: pip install catboost") from e
    return CatBoostClassifier(iterations=iterations, learning_rate=0.05, depth=6,
                              verbose=MODEL_VERBOSE, random_seed=RANDOM_STATE, thread_count=-1)

# -------------------- Fitness cache --------------------
# key: tuple(selected original indices) -> float score
fitness_cache = {}

def key_from_mask(mask_bool):
    return tuple(sorted(np.where(np.array(mask_bool).astype(bool))[0].tolist()))

def evaluate_mask_global(mask_bool, cv=CV_OPT, cb_iter=CB_ITER_OPT):
    """
    Evaluate mask using CatBoost with CV and return average of acc,prec,rec,f1.
    Caches results to avoid re-evaluating identical subsets.
    """
    key = key_from_mask(mask_bool)
    if key in fitness_cache:
        return fitness_cache[key]
    if len(key) == 0:
        fitness_cache[key] = 0.0
        return 0.0

    X_sel = X.iloc[:, list(key)]
    model = get_catboost_model(iterations=cb_iter)
    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=RANDOM_STATE)

    accs = cross_val_score(clone(model), X_sel, y, cv=skf, scoring="accuracy", n_jobs=-1)
    precs = cross_val_score(clone(model), X_sel, y, cv=skf, scoring=make_scorer(precision_score, zero_division=0), n_jobs=-1)
    recs = cross_val_score(clone(model), X_sel, y, cv=skf, scoring=make_scorer(recall_score, zero_division=0), n_jobs=-1)
    f1s = cross_val_score(clone(model), X_sel, y, cv=skf, scoring=make_scorer(f1_score, zero_division=0), n_jobs=-1)

    score = float((np.mean(accs) + np.mean(precs) + np.mean(recs) + np.mean(f1s)) / 4.0)
    fitness_cache[key] = score
    return score

# -------------------- Helpers --------------------
def mask_to_features(mask):
    idxs = np.where(np.array(mask).astype(bool))[0].tolist()
    return [FEATURE_NAMES[i] for i in idxs]

def log(msg):
    print(f"[{time.strftime('%H:%M:%S')}] {msg}", flush=True)

# -------------------- PSO (binary) --------------------
def run_pso(swarm_size=PSO_SWARM, iters=PSO_ITERS, cv=CV_OPT):
    log(f"PSO START (swarm={swarm_size}, iters={iters}, cv={cv})")
    t0 = time.time()
    dim = N_FEATURES
    pos = np.random.randint(0,2,(swarm_size,dim)).astype(int)
    vel = np.random.uniform(-1,1,(swarm_size,dim))

    pbest = pos.copy()
    pbest_scores = np.array([evaluate_mask_global(p.astype(bool), cv=cv, cb_iter=CB_ITER_OPT) for p in pos])

    gbest_idx = int(np.argmax(pbest_scores))
    gbest = pbest[gbest_idx].copy()
    gbest_score = pbest_scores[gbest_idx]

    w = 0.6; c1 = c2 = 1.5
    for t in range(iters):
        log(f" PSO iter {t+1}/{iters} best_global={gbest_score:.4f}")
        for i in range(swarm_size):
            r1 = np.random.rand(dim); r2 = np.random.rand(dim)
            vel[i] = w*vel[i] + c1*r1*(pbest[i] - pos[i]) + c2*r2*(gbest - pos[i])
            s = 1.0 / (1.0 + np.exp(-vel[i]))
            pos[i] = (np.random.rand(dim) < s).astype(int)

            sc = evaluate_mask_global(pos[i].astype(bool), cv=cv, cb_iter=CB_ITER_OPT)
            if sc > pbest_scores[i]:
                pbest[i] = pos[i].copy()
                pbest_scores[i] = sc
            if sc > gbest_score:
                gbest = pos[i].copy()
                gbest_score = sc
        w = max(0.2, w*0.97)

    best_idx = int(np.argmax(pbest_scores))
    best_mask = pbest[best_idx].copy()
    best_score = pbest_scores[best_idx]
    t1 = time.time()
    log(f"PSO DONE in {int(t1-t0)}s best_score={best_score:.4f} selected={int(np.sum(best_mask))}")
    log(f"PSO SELECTED FEATURES: {mask_to_features(best_mask)}")

    return best_mask, best_score, int(t1-t0)

# -------------------- GA (binary) --------------------
def run_ga(pop_size=GA_POP, gens=GA_GENS, cv=CV_OPT):
    log(f"GA START (pop={pop_size}, gens={gens}, cv={cv})")
    t0 = time.time()
    dim = N_FEATURES
    pop = np.random.randint(0,2,(pop_size, dim)).astype(int)
    fitness_scores = np.array([evaluate_mask_global(ind.astype(bool), cv=cv, cb_iter=CB_ITER_OPT) for ind in pop])

    def tournament_select(k=3):
        idxs = np.random.randint(0, pop_size, k)
        return idxs[np.argmax(fitness_scores[idxs])]

    for g in range(gens):
        log(f" GA gen {g+1}/{gens} current_best={np.max(fitness_scores):.4f}")
        new_pop = []
        # elitism
        elite_idxs = np.argsort(fitness_scores)[-2:]
        new_pop.extend(pop[elite_idxs].tolist())

        while len(new_pop) < pop_size:
            i1 = tournament_select(); i2 = tournament_select()
            p1 = pop[i1].copy(); p2 = pop[i2].copy()
            # crossover
            if np.random.rand() < 0.7:
                pt = np.random.randint(1, dim)
                c1 = np.concatenate([p1[:pt], p2[pt:]])
                c2 = np.concatenate([p2[:pt], p1[pt:]])
            else:
                c1, c2 = p1, p2
            # mutation
            for child in (c1, c2):
                for d in range(dim):
                    if np.random.rand() < 0.1:
                        child[d] = 1 - child[d]
                new_pop.append(child)
                if len(new_pop) >= pop_size:
                    break
        pop = np.array(new_pop[:pop_size])
        fitness_scores = np.array([evaluate_mask_global(ind.astype(bool), cv=cv, cb_iter=CB_ITER_OPT) for ind in pop])

    best_idx = int(np.argmax(fitness_scores))
    best_mask = pop[best_idx].copy()
    best_score = fitness_scores[best_idx]
    t1 = time.time()
    log(f"GA DONE in {int(t1-t0)}s best_score={best_score:.4f} selected={int(np.sum(best_mask))}")
    log(f"GA SELECTED FEATURES: {mask_to_features(best_mask)}")

    return best_mask, best_score, int(t1-t0)

# -------------------- GWO (binary) --------------------
def run_gwo(wolves=GWO_WOLVES, iters=GWO_ITERS, cv=CV_OPT):
    log(f"GWO START (wolves={wolves}, iters={iters}, cv={cv})")
    t0 = time.time()
    dim = N_FEATURES
    pop = np.random.randint(0,2,(wolves, dim)).astype(int)
    fitness_scores = np.array([evaluate_mask_global(ind.astype(bool), cv=cv, cb_iter=CB_ITER_OPT) for ind in pop])

    Alpha = Beta = Delta = None
    Alpha_score = Beta_score = Delta_score = -1.0

    for itr in range(iters):
        log(f" GWO iter {itr+1}/{iters} best_alpha={Alpha_score:.4f}")
        for i in range(wolves):
            sc = fitness_scores[i]
            if sc > Alpha_score:
                Delta_score, Beta_score, Alpha_score = Beta_score, Alpha_score, sc
                Delta, Beta, Alpha = Beta, Alpha, pop[i].copy()
            elif sc > Beta_score:
                Delta_score, Beta_score = Beta_score, sc
                Delta, Beta = Beta, pop[i].copy()
            elif sc > Delta_score:
                Delta_score = sc
                Delta = pop[i].copy()

        a = 2 - itr * (2.0 / iters)
        for i in range(wolves):
            for d in range(dim):
                if Alpha is None:
                    continue
                r1, r2 = np.random.rand(), np.random.rand()
                A1 = 2 * a * r1 - a; C1 = 2 * r2
                D_alpha = abs(C1 * Alpha[d] - pop[i][d])
                X1 = Alpha[d] - A1 * D_alpha

                r1, r2 = np.random.rand(), np.random.rand()
                A2 = 2 * a * r1 - a; C2 = 2 * r2
                D_beta = abs(C2 * Beta[d] - pop[i][d])
                X2 = Beta[d] - A2 * D_beta

                r1, r2 = np.random.rand(), np.random.rand()
                A3 = 2 * a * r1 - a; C3 = 2 * r2
                D_delta = abs(C3 * Delta[d] - pop[i][d])
                X3 = Delta[d] - A3 * D_delta

                new_pos = (X1 + X2 + X3) / 3.0
                s = 1.0 / (1.0 + np.exp(-new_pos))
                pop[i][d] = 1 if np.random.rand() < s else 0

        fitness_scores = np.array([evaluate_mask_global(ind.astype(bool), cv=cv, cb_iter=CB_ITER_OPT) for ind in pop])

    best_idx = int(np.argmax(fitness_scores))
    best_mask = pop[best_idx].copy()
    best_score = fitness_scores[best_idx]
    t1 = time.time()
    log(f"GWO DONE in {int(t1-t0)}s best_score={best_score:.4f} selected={int(np.sum(best_mask))}")
    log(f"GWO SELECTED FEATURES: {mask_to_features(best_mask)}")

    return best_mask, best_score, int(t1-t0)

# -------------------- INTERSECTION / UNION / VOTING --------------------
def get_intersection_mask(*masks):
    """Return mask that contains only features present in ALL provided masks."""
    if len(masks) == 0:
        return np.zeros(N_FEATURES, dtype=int)
    inter_idx = set(np.where(np.array(masks[0]).astype(bool))[0].tolist())
    for m in masks[1:]:
        idxs = set(np.where(np.array(m).astype(bool))[0].tolist())
        inter_idx = inter_idx.intersection(idxs)
    mask = np.zeros(N_FEATURES, dtype=int)
    for i in inter_idx:
        mask[i] = 1
    return mask


def get_union_mask(*masks):
    union_idx = set()
    for m in masks:
        idxs = np.where(np.array(m).astype(bool))[0].tolist()
        union_idx.update(idxs)
    mask = np.zeros(N_FEATURES, dtype=int)
    for i in union_idx:
        mask[i] = 1
    return mask


def get_voting_mask(*masks, threshold=2):
    """Return mask of features selected by at least `threshold` methods (default majority of 3 => 2)."""
    if len(masks) == 0:
        return np.zeros(N_FEATURES, dtype=int)
    counts = np.zeros(N_FEATURES, dtype=int)
    for m in masks:
        counts += np.array(m).astype(int)
    mask = (counts >= threshold).astype(int)
    return mask

# -------------------- HLO on candidates --------------------
def hlo_on_candidates(candidate_mask, pop_size=HLO_POP, iters=HLO_ITERS, cv=CV_OPT):
    candidate_indices = np.where(np.array(candidate_mask).astype(bool))[0].tolist()
    k = len(candidate_indices)
    if k == 0:
        raise ValueError("Candidate set is empty.")

    log(f"HLO START on {k} candidate features (pop={pop_size}, iters={iters})")
    t0 = time.time()

    pop = np.random.randint(0,2,(pop_size, k)).astype(int)

    def fitness_candidate(bitmask):
        full_mask = np.zeros(N_FEATURES, dtype=int)
        for j,bit in enumerate(bitmask):
            if bit == 1:
                full_mask[candidate_indices[j]] = 1
        return evaluate_mask_global(full_mask.astype(bool), cv=cv, cb_iter=CB_ITER_HLO)

    fitness_scores = np.array([fitness_candidate(ind) for ind in pop])
    best_idx = int(np.argmax(fitness_scores))
    best_solution = pop[best_idx].copy()
    best_score = fitness_scores[best_idx]

    for it in range(iters):
        log(f" HLO iter {it+1}/{iters} current_best={best_score:.4f}")
        teacher = pop[int(np.argmax(fitness_scores))].copy()
        new_pop = []
        for i in range(pop_size):
            learner = pop[i].copy()
            # teaching phase
            for d in range(k):
                if np.random.rand() < HLO_TEACHER_FACTOR:
                    learner[d] = teacher[d]
            # peer learning
            partner = pop[np.random.randint(pop_size)].copy()
            for d in range(k):
                if learner[d] != partner[d] and np.random.rand() < 0.5:
                    learner[d] = partner[d]
            # mutation
            for d in range(k):
                if np.random.rand() < HLO_MUTATION:
                    learner[d] = 1 - learner[d]
            new_pop.append(learner)
        pop = np.array(new_pop)
        fitness_scores = np.array([fitness_candidate(ind) for ind in pop])
        gen_best_idx = int(np.argmax(fitness_scores))
        gen_best_score = fitness_scores[gen_best_idx]
        gen_best_sol = pop[gen_best_idx].copy()
        if gen_best_score > best_score:
            best_score = gen_best_score
            best_solution = gen_best_sol.copy()

    # map back to full mask
    final_full_mask = np.zeros(N_FEATURES, dtype=int)
    for j,bit in enumerate(best_solution):
        if bit == 1:
            final_full_mask[candidate_indices[j]] = 1

    t1 = time.time()
    log(f"HLO DONE in {int(t1-t0)}s best_score={best_score:.4f} final_selected={int(np.sum(final_full_mask))}")
    return final_full_mask, best_score, int(t1-t0)

# -------------------- Greedy Hill-Climb (local search) --------------------
def hill_climb_on_candidates(initial_mask, candidate_mask, max_steps=HILLCLIMB_MAX_STEPS, eval_cap=HILLCLIMB_EVAL_CAP, cv=CV_OPT):
    """
    Greedy single-bit flip hill-climb restricted to candidate indices.
    Starts from initial_mask (full-length). Tries flipping each candidate feature's bit:
    - If flip improves fitness, accept and restart scanning.
    - Stops when no improving flip found or max_steps/eval_cap reached.
    """
    candidate_indices = np.where(np.array(candidate_mask).astype(bool))[0].tolist()
    if len(candidate_indices) == 0:
        log("Hill-climb: candidate set empty, skipping.")
        return initial_mask, 0.0, 0

    log(f"Hill-climb START over {len(candidate_indices)} candidates (max_steps={max_steps}, eval_cap={eval_cap})")
    t0 = time.time()
    current_mask = initial_mask.copy()
    current_score = evaluate_mask_global(current_mask.astype(bool), cv=cv, cb_iter=CB_ITER_HLO)
    evals = 0
    steps = 0
    improved = True

    while improved and steps < max_steps and evals < eval_cap:
        improved = False
        for idx in np.random.permutation(candidate_indices):
            trial_mask = current_mask.copy()
            trial_mask[idx] = 1 - trial_mask[idx]  # flip
            trial_score = evaluate_mask_global(trial_mask.astype(bool), cv=cv, cb_iter=CB_ITER_HLO)
            evals += 1
            if trial_score > current_score + 1e-8:
                current_mask = trial_mask
                current_score = trial_score
                improved = True
                steps += 1
                log(f" Hill-climb step {steps}: flipped {FEATURE_NAMES[idx]} -> new_score={current_score:.4f} (evals={evals})")
                break
            if evals >= eval_cap or steps >= max_steps:
                break
    t1 = time.time()
    log(f"Hill-climb DONE in {int(t1-t0)}s steps={steps} evals={evals} final_score={current_score:.4f} selected={int(np.sum(current_mask))}")
    return current_mask, current_score, int(t1-t0)

# -------------------- Final evaluation (5-fold CV) --------------------
def final_evaluation(mask_bool, cv=CV_FINAL, cb_iter=CB_ITER_FINAL):
    idxs = np.where(np.array(mask_bool).astype(bool))[0].tolist()
    if len(idxs) == 0:
        raise ValueError("Final mask selects zero features.")
    X_sel = X.iloc[:, idxs]
    model = get_catboost_model(iterations=cb_iter)
    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=RANDOM_STATE)
    accs = []; precs = []; recs = []; f1s = []
    t0 = time.time()
    for tr,te in skf.split(X_sel, y):
        m = clone(model); m.fit(X_sel.iloc[tr], y.iloc[tr])
        pred = m.predict(X_sel.iloc[te])
        accs.append(accuracy_score(y.iloc[te], pred))
        precs.append(precision_score(y.iloc[te], pred, zero_division=0))
        recs.append(recall_score(y.iloc[te], pred, zero_division=0))
        f1s.append(f1_score(y.iloc[te], pred, zero_division=0))
    t1 = time.time()
    results = {
        "n_features": len(idxs),
        "features": [FEATURE_NAMES[i] for i in idxs],
        "acc_mean": float(np.mean(accs)), "acc_std": float(np.std(accs)),
        "prec_mean": float(np.mean(precs)), "prec_std": float(np.std(precs)),
        "rec_mean": float(np.mean(recs)), "rec_std": float(np.std(recs)),
        "f1_mean": float(np.mean(f1s)), "f1_std": float(np.std(f1s)),
        "eval_time_s": int(t1 - t0)
    }
    return results

# -------------------- MAIN PIPELINE --------------------
if __name__ == "__main__":
    total_t0 = time.time()
    log("===== HYBRID (reduced budget) + HLO + HILL-CLIMB (UNION/INTERSECTION/VOTING) START =====")

    # PSO
    pso_mask, pso_score, pso_time = run_pso(swarm_size=PSO_SWARM, iters=PSO_ITERS, cv=CV_OPT)

    # GA
    ga_mask, ga_score, ga_time = run_ga(pop_size=GA_POP, gens=GA_GENS, cv=CV_OPT)

    # GWO
    gwo_mask, gwo_score, gwo_time = run_gwo(wolves=GWO_WOLVES, iters=GWO_ITERS, cv=CV_OPT)

    # Derive candidate masks
    union_mask = get_union_mask(pso_mask, ga_mask, gwo_mask)
    inter_mask = get_intersection_mask(pso_mask, ga_mask, gwo_mask)
    vote_mask = get_voting_mask(pso_mask, ga_mask, gwo_mask, threshold=2)

    candidate_sets = {
        'union': union_mask,
        'intersection': inter_mask,
        'voting': vote_mask
    }

    results_all = {}

    # run HLO -> hill-climb -> final evaluation -> train & save model for each candidate set
    for name, cand_mask in candidate_sets.items():
        log(f"===== PROCESSING {name.upper()} CANDIDATES =====")
        n_cand = int(np.sum(cand_mask))
        log(f"{name.upper()} candidate features: {n_cand}")
        if n_cand == 0:
            log(f"{name.upper()} empty — skipping HLO/hill-climb and model training.")
            results_all[name] = {'skipped': True, 'n_candidates': 0}
            continue

        # HLO on this candidate set
        hlo_mask, hlo_score, hlo_time = hlo_on_candidates(cand_mask, pop_size=HLO_POP, iters=HLO_ITERS, cv=CV_OPT)

        # hill-climb restricted to candidate set
        hc_mask, hc_score, hc_time = hill_climb_on_candidates(hlo_mask, cand_mask, max_steps=HILLCLIMB_MAX_STEPS, eval_cap=HILLCLIMB_EVAL_CAP, cv=CV_OPT)

        # final CV evaluation
        final_res = final_evaluation(hc_mask, cv=CV_FINAL, cb_iter=CB_ITER_FINAL)

        # Train final CatBoost model on 80% train and evaluate on 20% test (stratified)
        sel_idxs = np.where(np.array(hc_mask).astype(bool))[0].tolist()
        sel_features = [FEATURE_NAMES[i] for i in sel_idxs]

        if len(sel_features) == 0:
            log(f"No features selected after hill-climb for {name}, skipping model train.")
            results_all[name] = {'skipped': True, 'n_candidates': n_cand}
            continue

        X_sel = X[sel_features]
        X_train, X_test, y_train, y_test = train_test_split(X_sel, y, test_size=FINAL_TEST_SIZE, stratify=y, random_state=RANDOM_STATE)

        model = get_catboost_model(iterations=CB_ITER_FINAL)
        model.fit(X_train, y_train)

        # evaluate on held-out test set (20%)
        y_pred = model.predict(X_test)
        test_acc = accuracy_score(y_test, y_pred)
        test_prec = precision_score(y_test, y_pred, zero_division=0)
        test_rec = recall_score(y_test, y_pred, zero_division=0)
        test_f1 = f1_score(y_test, y_pred, zero_division=0)

        test_metrics = {
            'acc': float(test_acc), 'prec': float(test_prec), 'rec': float(test_rec), 'f1': float(test_f1),
            'n_test': int(X_test.shape[0])
        }

        # Save model to file (pickle)
        model_filename = f"{SAVE_PREFIX}_{name}_model.pkl"
        with open(model_filename, 'wb') as mf:
            pickle.dump(model, mf)

        # store results
        results_all[name] = {
            'n_candidates': n_cand,
            'hlo_score': float(hlo_score), 'hlo_time': int(hlo_time),
            'hc_score': float(hc_score), 'hc_time': int(hc_time),
            'final_eval': final_res,
            'selected_features': sel_features,
            'model_file': model_filename,
            'test_metrics': test_metrics
        }

        log(f"Saved trained CatBoost model for {name} -> {model_filename} (test_f1={test_f1:.4f})")

    total_t1 = time.time()
    elapsed_total = int(total_t1 - total_t0)

    # Summary / save aggregated results
    print("==================== AGGREGATE SUMMARY ====================")
    print(f"PSO  -> opt_score={pso_score:.4f} selected={int(np.sum(pso_mask))} time={pso_time}s")
    print(f"GA   -> opt_score={ga_score:.4f} selected={int(np.sum(ga_mask))} time={ga_time}s")
    print(f"GWO  -> opt_score={gwo_score:.4f} selected={int(np.sum(gwo_mask))} time={gwo_time}s")
    print(f"Union candidates    : {int(np.sum(union_mask))}")
    print(f"Intersection candidates: {int(np.sum(inter_mask))}")
    print(f"Voting candidates   : {int(np.sum(vote_mask))}")
    print("-------------------------------------------------")

    for name, info in results_all.items():
        print(f"-- {name.upper()} SUMMARY --")
        if info.get('skipped'):
            print(" skipped (no candidates)")
            continue
        fe = info['final_eval']
        tm = info['test_metrics']
        print(f" Selected ({fe['n_features']}): {fe['features']}")
        print(f" CV F1   : {fe['f1_mean']:.4f} ± {fe['f1_std']:.4f}")
        print(f" Test F1 : {tm['f1']:.4f} (n_test={tm['n_test']})")
        print(f" Accuracy : {fe['acc_mean']:.4f} ± {fe['acc_std']:.4f}")
        print(f" Precision: {fe['prec_mean']:.4f} ± {fe['prec_std']:.4f}")
        print(f" Recall   : {fe['rec_mean']:.4f} ± {fe['rec_std']:.4f}")
        print(f" Model file: {info['model_file']}")



    # Save aggregated pipeline outputs
    out = {
        "pso_mask": pso_mask, "pso_score": pso_score, "pso_time": pso_time,
        "ga_mask": ga_mask, "ga_score": ga_score, "ga_time": ga_time,
        "gwo_mask": gwo_mask, "gwo_score": gwo_score, "gwo_time": gwo_time,
        "union_mask": union_mask, "intersection_mask": inter_mask, "voting_mask": vote_mask,
        "results_all": results_all,
        "fitness_cache_len": len(fitness_cache)
    }
    with open(f"{SAVE_PREFIX}_results.pkl", "wb") as f:
        pickle.dump(out, f)

    log(f"Saved results to {SAVE_PREFIX}_results.pkl")
    log("===== PIPELINE COMPLETE =====")

In [70]:
import pickle
import pandas as pd

# Choose which model to test
flow = "union"               # "intersection" or "voting"

# Load results dictionary (contains selected features for every flow)
results_path = "/kaggle/working/hybrid_hlo_models_results.pkl"

with open(results_path, "rb") as f:
    results_all = pickle.load(f)

# Get selected features
selected_features = results_all["results_all"][flow]["selected_features"]
print("Selected features for", flow, ":", selected_features)


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/hybrid_hlo_models_results.pkl'

In [ ]:
model_path = f"/kaggle/working/hybrid_hlo_models_{flow}_model.pkl"

with open(model_path, "rb") as f:
    model = pickle.load(f)

print("Model loaded:", model)


In [68]:
df2017 = pd.read_parquet("/kaggle/input/cicids2017/Botnet-Friday-no-metadata.parquet")

print("Columns in 2017 dataset:", df2017.columns.tolist())


Columns in 2017 dataset: ['Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Fwd Packets Length Total', 'Bwd Packets Length Total', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'CWE Flag Count', 'ECE

In [69]:
# Check missing features
missing = [f for f in selected_features if f not in df2017.columns]

if missing:
    print("❌ Missing in 2017 dataset:", missing)
    raise ValueError("Dataset missing required features.")

# Create X_test
X_test_2017 = df2017[selected_features]

# Extract y if present
if "Label" in df2017.columns:
    y_test_2017 = df2017["Label"].astype(int)
else:
    y_test_2017 = None


NameError: name 'selected_features' is not defined

In [ ]:
y_pred_2017 = model.predict(X_test_2017)

if y_test_2017 is not None:
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

    acc  = accuracy_score(y_test_2017, y_pred_2017)
    prec = precision_score(y_test_2017, y_pred_2017, zero_division=0)
    rec  = recall_score(y_test_2017, y_pred_2017, zero_division=0)
    f1   = f1_score(y_test_2017, y_pred_2017, zero_division=0)

    print(f"\n=== {flow.upper()} model results on 2017 dataset ===")
    print("Accuracy :", acc)
    print("Precision:", prec)
    print("Recall   :", rec)
    print("F1 Score :", f1)
else:
    print("Predictions:", y_pred_2017[:20])


In [ ]:
# Troubleshoot missing features and optionally auto-map close matches
import pickle, difflib, pandas as pd
from pprint import pprint

# ---------- paths (adjust if needed) ----------
results_path = "/kaggle/working/hybrid_hlo_models_results.pkl"
flow = "union"   # change to "intersection" or "voting" as needed
model_pkl_path = f"/kaggle/working/hybrid_hlo_models_{flow}_model.pkl"
parquet_path = "/kaggle/input/cicids2017/Botnet-Friday-no-metadata.parquet"  # change if needed

# ---------- load saved results to get the selected feature list ----------
with open(results_path, "rb") as f:
    top = pickle.load(f)

selected_features = top["results_all"][flow]["selected_features"]
print(f"Model expects {len(selected_features)} features for '{flow}':")
pprint(selected_features)
print()

# ---------- load 2017 dataset (parquet) ----------
df2017 = pd.read_parquet(parquet_path)
print("2017 dataset columns (count):", len(df2017.columns))
# print(df2017.columns.tolist())  # uncomment if you want the full list

# ---------- check which are missing ----------
available_cols = list(df2017.columns)
missing = [f for f in selected_features if f not in available_cols]

if not missing:
    print("All required features present — you can build X_test directly.")
    X_test_2017 = df2017[selected_features]
else:
    print(f"Missing {len(missing)} features (exact match):")
    pprint(missing)
    print("\nNow showing close matches for each missing feature (top 3 suggestions):\n")
    for m in missing:
        # find close matches among available_cols
        matches = difflib.get_close_matches(m, available_cols, n=3, cutoff=0.6)
        print(f" -> {m}  => suggestions: {matches}")

    # ---------- OPTIONAL: attempt automatic mapping ----------
    AUTO_MAP = True   # set False to not attempt mapping
    if AUTO_MAP:
        mapping = {}   # model_feature -> dataset_column
        used_dataset_cols = set()

        for m in missing:
            # try exact case-insensitive match first
            ci = [c for c in available_cols if c.lower() == m.lower()]
            if ci:
                mapping[m] = ci[0]
                used_dataset_cols.add(ci[0])
                continue

            # otherwise use difflib suggestions (cutoff tuned)
            suggs = difflib.get_close_matches(m, available_cols, n=1, cutoff=0.75)
            if suggs:
                mapping[m] = suggs[0]
                used_dataset_cols.add(suggs[0])
            else:
                # try relaxed matching: strip spaces, underscores, lower-case
                norm_m = m.replace(" ", "").replace("_", "").lower()
                best = None; best_score = 0.0
                for c in available_cols:
                    score = difflib.SequenceMatcher(None, norm_m, c.replace(" ", "").replace("_","").lower()).ratio()
                    if score > best_score:
                        best_score = score; best = c
                if best_score >= 0.7:
                    mapping[m] = best
                    used_dataset_cols.add(best)

        print("\nAuto-mapping proposals (model_feature -> dataset_column):")
        pprint(mapping)

        # show which missing remain after mapping
        still_missing = [m for m in missing if m not in mapping]
        if still_missing:
            print("\nStill missing (not automatically mapped):")
            pprint(still_missing)
            raise ValueError("Automatic mapping incomplete — please inspect suggestions above and map manually.")
        else:
            # Build X_test using mapped names and existing exact-matched ones
            final_cols = []
            for feat in selected_features:
                if feat in available_cols:
                    final_cols.append(feat)
                else:
                    # mapped
                    final_cols.append(mapping[feat])
            print("\nFinal column list to use (in correct order):")
            pprint(final_cols)

            # Optional: show any duplicates or collisions
            if len(final_cols) != len(set(final_cols)):
                print("\nWarning: mapped column list contains duplicates (some model features map to the same dataset column).")
                dupes = [c for c in final_cols if final_cols.count(c) > 1]
                pprint(list(set(dupes)))
                raise ValueError("Duplicate mappings found — resolve mapping ambiguity before proceeding.")

            # Build X_test_2017
            X_test_2017 = df2017[final_cols].copy()
            print("\nX_test_2017 built successfully with shape:", X_test_2017.shape)
            # If Label exists, provide y as well
            y_test_2017 = df2017["Label"].astype(int) if "Label" in df2017.columns else None

            # You can now predict:
            # with open(model_pkl_path, "rb") as mf:
            #     model = pickle.load(mf)
            # preds = model.predict(X_test_2017)
            # ... evaluate as needed ...


In [ ]:
# Run in Kaggle notebook
import pandas as pd
import numpy as np
import difflib
from pathlib import Path

# ---------- EDIT THIS PATH IF NEEDED ----------
parquet_path = "/kaggle/input/cicids2017/Botnet-Friday-no-metadata.parquet"
out_dir = "/kaggle/working"
Path(out_dir).mkdir(parents=True, exist_ok=True)

# target columns (same order as you requested)
target_cols = [
  'Dst Port','Timestamp','Flow Duration','Fwd Pkt Len Min','Fwd Pkt Len Mean',
  'Bwd Pkt Len Max','Bwd Pkt Len Min','Bwd Pkt Len Std','Flow Pkts/s','Fwd IAT Std',
  'Fwd Header Len','Fwd Pkts/s','Pkt Len Mean','Pkt Len Var','SYN Flag Cnt',
  'CWE Flag Count','ECE Flag Cnt','Down/Up Ratio','Subflow Bwd Byts',
  'Init Bwd Win Byts','Fwd Seg Size Min','Idle Mean','Idle Std','Idle Min',
  'Src IP','Dst IP'
]

# initial candidate map (target -> suggested source name based on earlier mapping)
candidate_map = {
  'Dst Port': None,
  'Timestamp': None,
  'Flow Duration': 'Flow Duration',
  'Fwd Pkt Len Min': 'Fwd Packet Length Min',
  'Fwd Pkt Len Mean': 'Fwd Packet Length Mean',
  'Bwd Pkt Len Max': 'Bwd Packet Length Max',
  'Bwd Pkt Len Min': 'Bwd Packet Length Min',
  'Bwd Pkt Len Std': 'Bwd Packet Length Std',
  'Flow Pkts/s': 'Flow Packets/s',
  'Fwd IAT Std': 'Fwd IAT Std',
  'Fwd Header Len': 'Fwd Header Length',
  'Fwd Pkts/s': 'Fwd Packets/s',
  'Pkt Len Mean': 'Packet Length Mean',
  'Pkt Len Var': 'Packet Length Variance',
  'SYN Flag Cnt': 'SYN Flag Count',
  'CWE Flag Count': 'CWE Flag Count',
  'ECE Flag Cnt': 'ECE Flag Count',
  'Down/Up Ratio': 'Down/Up Ratio',
  'Subflow Bwd Byts': 'Subflow Bwd Bytes',
  'Init Bwd Win Byts': 'Init Bwd Win Bytes',
  'Fwd Seg Size Min': 'Fwd Seg Size Min',
  'Idle Mean': 'Idle Mean',
  'Idle Std': 'Idle Std',
  'Idle Min': 'Idle Min',
  'Src IP': None,
  'Dst IP': None
}

print("Loading parquet (this can take a few seconds)...")
df = pd.read_parquet(parquet_path)

src_cols = list(df.columns)
print(f"Source columns in parquet: {len(src_cols)} columns loaded.")

# Function: attempt to resolve mapping (exact -> candidate -> fuzzy)
def resolve_mapping(candidate_map, src_cols, fuzzy_cutoff=0.7):
    final_map = {}
    fuzzy_suggestions = {}
    for tgt, guess in candidate_map.items():
        resolved = None
        # 1) exact match (case sensitive)
        if guess and guess in src_cols:
            resolved = guess
        # 2) try normalized exact (lower, remove spaces/punctuation)
        if resolved is None and guess:
            norm_guess = guess.lower().replace(' ', '').replace('_','')
            for c in src_cols:
                if c.lower().replace(' ','').replace('_','') == norm_guess:
                    resolved = c
                    break
        # 3) try exact on target name (maybe target already exists)
        if resolved is None and tgt in src_cols:
            resolved = tgt
        # 4) fuzzy match: compare target and guess to source columns
        if resolved is None:
            # candidates: compare both target and guess strings to source column names
            keys_to_try = [tgt]
            if guess:
                keys_to_try.insert(0, guess)
            best_match = None
            best_ratio = 0.0
            for key in keys_to_try:
                matches = difflib.get_close_matches(key, src_cols, n=3, cutoff=fuzzy_cutoff)
                if matches:
                    # difflib doesn't give score; use SequenceMatcher for best of matches
                    from difflib import SequenceMatcher
                    for m in matches:
                        ratio = SequenceMatcher(None, key.lower(), m.lower()).ratio()
                        if ratio > best_ratio:
                            best_ratio = ratio
                            best_match = m
            if best_match and best_ratio >= fuzzy_cutoff:
                resolved = best_match
                fuzzy_suggestions[tgt] = (best_match, best_ratio)
        final_map[tgt] = resolved
    return final_map, fuzzy_suggestions

final_map, fuzzy_suggestions = resolve_mapping(candidate_map, src_cols, fuzzy_cutoff=0.66)

# Report mapping attempts
mapped = {t: s for t, s in final_map.items() if s}
unmapped = [t for t, s in final_map.items() if not s]

print("\n=== Mapping summary ===")
print("Mapped targets:")
for t, s in mapped.items():
    print(f"  {t}  <-  {s}")
if fuzzy_suggestions:
    print("\nFuzzy match suggestions (with approx. confidence):")
    for t, (m, r) in fuzzy_suggestions.items():
        print(f"  {t}: suggested '{m}' (score ≈ {r:.2f})")
if unmapped:
    print("\nUnmapped targets (will be created as empty columns):")
    for t in unmapped:
        print("  ", t)

# Apply renaming: construct rename dict (source_col -> target_col) for those resolved
rename_dict = {}
for tgt, src in final_map.items():
    if src:
        # rename source column name to the target column name
        rename_dict[src] = tgt

# Make a copy so we don't alter original variable usage
df2 = df.copy()

# Rename columns
if rename_dict:
    df2 = df2.rename(columns=rename_dict)
    print(f"\nRenamed {len(rename_dict)} columns.")

# Create missing columns with NaN (or appropriate placeholder)
for tgt in target_cols:
    if tgt not in df2.columns:
        df2[tgt] = np.nan

# Reorder columns to exactly match target_cols; if you want to keep other columns append them after
df_out = df2[target_cols].copy()

# Save output
out_parquet = Path(out_dir) / "processed_botnet_2017.parquet"
out_csv = Path(out_dir) / "processed_botnet_2017.csv"
df_out.to_parquet(out_parquet, index=False)
df_out.to_csv(out_csv, index=False)
print(f"\nSaved processed files:\n  {out_parquet}\n  {out_csv}")

# Final info
print("\nFinal DataFrame shape:", df_out.shape)
print("Columns (in order):")
print(df_out.columns.tolist())

# list rows where placeholders remain (i.e., newly created columns are all-NaN)
placeholder_cols = [c for c in target_cols if df_out[c].isna().all()]
if placeholder_cols:
    print("\nColumns that are placeholders (all NaN):")
    for c in placeholder_cols:
        print("  ", c)
else:
    print("\nNo placeholder columns — all target cols have values.")


In [ ]:
ff=pd.read_csv("/kaggle/working/processed_botnet_2017.csv")
ff.columns
ff.shape

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from catboost import CatBoostClassifier
import time
import pickle

# -----------------------------------------
# 1️⃣ Load cleaned dataset
# -----------------------------------------
df = pd.read_csv("")

# Ensure "Label" column exists
if "Label" not in df.columns:
    raise ValueError("❌ ERROR: 'Label' column not found in dataset!")

# -----------------------------------------
# 2️⃣ Split features and target
# -----------------------------------------
X = df.drop("Label", axis=1)
y = df["Label"]

# -----------------------------------------
# 3️⃣ Train-test split (80-20)
# -----------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

# -----------------------------------------
# 4️⃣ Train CatBoost model + record time
# -----------------------------------------
model = CatBoostClassifier(
    iterations=300,
    learning_rate=0.1,
    depth=8,
    loss_function='MultiClass',
    verbose=False
)

start_time = time.time()
model.fit(X_train, y_train)
end_time = time.time()
training_time = end_time - start_time

# -----------------------------------------
# 5️⃣ Predictions
# -----------------------------------------
y_pred = model.predict(X_test)
y_pred = y_pred.flatten()

# -----------------------------------------
# 6️⃣ Metrics
# -----------------------------------------
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average="weighted")
recall = recall_score(y_test, y_pred, average="weighted")
precision = precision_score(y_test, y_pred, average="weighted")

# -----------------------------------------
# 7️⃣ Save model to Pickle
# -----------------------------------------
pickle_path = "/kaggle/working/catboost_ids2018_model.pkl"
with open(pickle_path, "wb") as f:
    pickle.dump(model, f)

# -----------------------------------------
# 8️⃣ Print results
# -----------------------------------------
print("✔️ CatBoost Training Complete\n")
print("📁 Model saved as:", pickle_path)
print("📌 Accuracy:", acc)
print("📌 F1 Score (weighted):", f1)
print("📌 Recall (weighted):", recall)
print("📌 Precision (weighted):", precision)
print(f"⏱️ Time Taken to Train: {training_time:.2f} seconds")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# ---------- 1. Load (avoid low-memory dtype guessing) ----------
input_path = "/kaggle/working/processed_botnet_2017_2.parquet"
df = pd.read_parquet(input_path)   # prevents chunked dtype inference warning

# Quick info to debug if needed
print("Initial shape:", df.shape)
print("Initial dtypes (sample):\n", df.dtypes.head(20))


# Drop exact duplicate rows
df = df.drop_duplicates().reset_index(drop=True)

# ---------- 3. Detect candidate numeric columns robustly ----------
# We'll try to coerce each column to numeric and keep columns that convert well.
numeric_candidates = []
conversion_stats = {}
for col in df.columns:
    # try converting to numeric (coerce errors -> NaN)
    coerced = pd.to_numeric(df[col], errors="coerce")
    non_na_ratio = coerced.notna().sum() / len(coerced)
    conversion_stats[col] = non_na_ratio
    # if at least 80% of values convert to numeric, treat as numeric (you can adjust threshold)
    if non_na_ratio >= 0.80:
        numeric_candidates.append(col)

print(f"Detected {len(numeric_candidates)} numeric-like columns (>=80% convertible).")

# Force convert those columns to numeric (float64). Others kept as-is (likely categorical/text)
for col in numeric_candidates:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# ---------- 4. Inspect for infinities or extreme values ----------
# Columns that contain ±inf
inf_cols = []
for col in numeric_candidates:
    ser = df[col]
    if np.isinf(ser.to_numpy()).any():
        inf_cols.append(col)

print("Columns with ±inf values:", inf_cols)

# Replace ±inf with NaN so we can handle them with median filling
if inf_cols:
    df[numeric_candidates] = df[numeric_candidates].replace([np.inf, -np.inf], np.nan)

# Check for huge values that might overflow float64 when operations applied
# We'll compute absolute max and print columns with extremely large values (> 1e300)
huge_cols = []
for col in numeric_candidates:
    try:
        max_abs = np.nanmax(np.abs(df[col].to_numpy()))
        if np.isfinite(max_abs) and max_abs > 1e300:
            huge_cols.append((col, max_abs))
    except Exception:
        # if conversion to numpy fails, skip
        pass

print("Columns with extremely large magnitudes (>1e300):", huge_cols)

# If you see columns in huge_cols you might want to inspect or clip them.
# For safety we'll clip numeric values to a sane range before scaling (optional)
CLIP_LIMIT = 1e300
df[numeric_candidates] = df[numeric_candidates].apply(lambda s: s.clip(lower=-CLIP_LIMIT, upper=CLIP_LIMIT))

# ---------- 5. Recompute numeric / categorical column lists ----------
num_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
print("Final numeric columns count:", len(num_cols))
print("Final categorical columns count:", len(cat_cols))

# ---------- 6. Handle missing values ----------
# Numeric: fill with median
if len(num_cols) > 0:
    df[num_cols] = df[num_cols].fillna(df[num_cols].median())

# Categorical: fill with mode (most frequent)
for col in cat_cols:
    if df[col].isna().any():
        mode_val = df[col].mode(dropna=True)
        if len(mode_val) > 0:
            df[col] = df[col].fillna(mode_val.iloc[0])
        else:
            # fallback to empty string if mode can't be computed
            df[col] = df[col].fillna("")

# ---------- 7. Encode categorical columns ----------
le = LabelEncoder()
for col in cat_cols:
    # convert to string first to avoid issues with mixed types
    df[col] = le.fit_transform(df[col].astype(str))

# ---------- 8. Final check for non-finite numerics before scaling ----------
# Replace any remaining inf/-inf with NaN (just in case), then fill with medians again
df[num_cols] = df[num_cols].replace([np.inf, -np.inf], np.nan)
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

# Ensure all numeric values are finite now
finite_check = {col: np.isfinite(df[col].to_numpy()).all() for col in num_cols}
bad = [c for c, ok in finite_check.items() if not ok]
print("Columns still non-finite (should be empty):", bad)

# ---------- 9. Min-Max scaling ----------
scaler = MinMaxScaler()
if len(num_cols) > 0:
    df[num_cols] = scaler.fit_transform(df[num_cols])

# ---------- 10. Save and report ----------
output_filename = "/kaggle/working/2019_cleaned_combined.csv"
df.to_csv(output_filename, index=False)

print("✅ Preprocessing complete!")
print("📁 Saved as:", output_filename)
print("Shape:", df.shape)
print("Label distribution (if present):")
if 'Label' in df.columns:
    print(df['Label'].value_counts())
else:
    print("No 'Label' column found in final dataframe.")


In [ ]:
df.columns
df.shape

In [ ]:
import pandas as pd

path = "/kaggle/working/2019_cleaned_combined.csv"
df = pd.read_csv(path)

# Normalize text (even if some trailing spaces exist)
df['Label'] = df['Label'].astype(str).str.strip().str.lower()

df['Label'] = df['Label'].apply(lambda x: 0 if x == "benign" else 1)

# Save back
df.to_csv("/kaggle/working/2019_cleaned_combined_binary.csv", index=False)

print("Label conversion complete!")
print(df['Label'].value_counts())


In [ ]:
# Run in Kaggle notebook (updated to retain 'Label' column)
import pandas as pd
import numpy as np
import difflib
from pathlib import Path

# ---------- EDIT THIS PATH IF NEEDED ----------
parquet_path = "/kaggle/input/cicids2017/Botnet-Friday-no-metadata.parquet"
out_dir = "/kaggle/working"
Path(out_dir).mkdir(parents=True, exist_ok=True)

# target columns (same order as you requested)
target_cols = [
  'Dst Port','Timestamp','Flow Duration','Fwd Pkt Len Min','Fwd Pkt Len Mean',
  'Bwd Pkt Len Max','Bwd Pkt Len Min','Bwd Pkt Len Std','Flow Pkts/s','Fwd IAT Std',
  'Fwd Header Len','Fwd Pkts/s','Pkt Len Mean','Pkt Len Var','SYN Flag Cnt',
  'CWE Flag Count','ECE Flag Cnt','Down/Up Ratio','Subflow Bwd Byts',
  'Init Bwd Win Byts','Fwd Seg Size Min','Idle Mean','Idle Std','Idle Min',
  'Src IP','Dst IP'
]

# initial candidate map (target -> suggested source name based on earlier mapping)
candidate_map = {
  'Dst Port': None,
  'Timestamp': None,
  'Flow Duration': 'Flow Duration',
  'Fwd Pkt Len Min': 'Fwd Packet Length Min',
  'Fwd Pkt Len Mean': 'Fwd Packet Length Mean',
  'Bwd Pkt Len Max': 'Bwd Packet Length Max',
  'Bwd Pkt Len Min': 'Bwd Packet Length Min',
  'Bwd Pkt Len Std': 'Bwd Packet Length Std',
  'Flow Pkts/s': 'Flow Packets/s',
  'Fwd IAT Std': 'Fwd IAT Std',
  'Fwd Header Len': 'Fwd Header Length',
  'Fwd Pkts/s': 'Fwd Packets/s',
  'Pkt Len Mean': 'Packet Length Mean',
  'Pkt Len Var': 'Packet Length Variance',
  'SYN Flag Cnt': 'SYN Flag Count',
  'CWE Flag Count': 'CWE Flag Count',
  'ECE Flag Cnt': 'ECE Flag Count',
  'Down/Up Ratio': 'Down/Up Ratio',
  'Subflow Bwd Byts': 'Subflow Bwd Bytes',
  'Init Bwd Win Byts': 'Init Bwd Win Bytes',
  'Fwd Seg Size Min': 'Fwd Seg Size Min',
  'Idle Mean': 'Idle Mean',
  'Idle Std': 'Idle Std',
  'Idle Min': 'Idle Min',
  'Src IP': None,
  'Dst IP': None
}

print("Loading parquet (this can take a few seconds)...")
df = pd.read_parquet(parquet_path)

src_cols = list(df.columns)
print(f"Source columns in parquet: {len(src_cols)} columns loaded.")

# Function: attempt to resolve mapping (exact -> candidate -> fuzzy)
def resolve_mapping(candidate_map, src_cols, fuzzy_cutoff=0.7):
    final_map = {}
    fuzzy_suggestions = {}
    for tgt, guess in candidate_map.items():
        resolved = None
        # 1) exact match (case sensitive)
        if guess and guess in src_cols:
            resolved = guess
        # 2) try normalized exact (lower, remove spaces/punctuation)
        if resolved is None and guess:
            norm_guess = guess.lower().replace(' ', '').replace('_','')
            for c in src_cols:
                if c.lower().replace(' ','').replace('_','') == norm_guess:
                    resolved = c
                    break
        # 3) try exact on target name (maybe target already exists)
        if resolved is None and tgt in src_cols:
            resolved = tgt
        # 4) fuzzy match: compare target and guess to source columns
        if resolved is None:
            # candidates: compare both target and guess strings to source column names
            keys_to_try = [tgt]
            if guess:
                keys_to_try.insert(0, guess)
            best_match = None
            best_ratio = 0.0
            for key in keys_to_try:
                matches = difflib.get_close_matches(key, src_cols, n=3, cutoff=fuzzy_cutoff)
                if matches:
                    # difflib doesn't give score; use SequenceMatcher for best of matches
                    from difflib import SequenceMatcher
                    for m in matches:
                        ratio = SequenceMatcher(None, key.lower(), m.lower()).ratio()
                        if ratio > best_ratio:
                            best_ratio = ratio
                            best_match = m
            if best_match and best_ratio >= fuzzy_cutoff:
                resolved = best_match
                fuzzy_suggestions[tgt] = (best_match, best_ratio)
        final_map[tgt] = resolved
    return final_map, fuzzy_suggestions

final_map, fuzzy_suggestions = resolve_mapping(candidate_map, src_cols, fuzzy_cutoff=0.66)

# Report mapping attempts
mapped = {t: s for t, s in final_map.items() if s}
unmapped = [t for t, s in final_map.items() if not s]

print("\n=== Mapping summary ===")
print("Mapped targets:")
for t, s in mapped.items():
    print(f"  {t}  <-  {s}")
if fuzzy_suggestions:
    print("\nFuzzy match suggestions (with approx. confidence):")
    for t, (m, r) in fuzzy_suggestions.items():
        print(f"  {t}: suggested '{m}' (score ≈ {r:.2f})")
if unmapped:
    print("\nUnmapped targets (will be created as empty columns):")
    for t in unmapped:
        print("  ", t)

# Apply renaming: construct rename dict (source_col -> target_col) for those resolved
rename_dict = {}
for tgt, src in final_map.items():
    if src:
        # don't rename into 'Label' accidentally (we keep original Label untouched)
        if src != 'Label':
            rename_dict[src] = tgt
        else:
            # if source column is literally 'Label' and target is not, keep as-is and warn
            print(f"Warning: source column 'Label' matched target '{tgt}'. Skipping rename for this mapping to preserve Label column.")

# Make a copy so we don't alter original variable usage
df2 = df.copy()

# Rename columns
if rename_dict:
    df2 = df2.rename(columns=rename_dict)
    print(f"\nRenamed {len(rename_dict)} columns.")

# Create missing columns with NaN (or appropriate placeholder)
# BUT do not overwrite an existing 'Label' column (if present)
for tgt in target_cols:
    if tgt not in df2.columns:
        df2[tgt] = np.nan

# Decide whether to include Label in output:
include_label = 'Label' in df.columns  # check original df to be safe

# Build final column order: all target_cols (in order) then Label (if it exists)
final_column_order = target_cols.copy()
if include_label:
    # If 'Label' is already among target_cols (unlikely), avoid duplication
    if 'Label' not in final_column_order:
        final_column_order.append('Label')

# Reorder columns to exactly match final_column_order; if you want to keep other columns append them after
# If some final columns are still not present (shouldn't happen except Label), create them as NaN earlier ensures presence
df_out = df2[final_column_order].copy()

# Save output
out_parquet = Path(out_dir) / "processed_botnet_2017_2.parquet"
out_csv = Path(out_dir) / "processed_botnet_2017.csv"
df_out.to_parquet(out_parquet, index=False)
df_out.to_csv(out_csv, index=False)
print(f"\nSaved processed files:\n  {out_parquet}\n  {out_csv}")

# Final info
print("\nFinal DataFrame shape:", df_out.shape)
print("Columns (in order):")
print(df_out.columns.tolist())

# list rows where placeholders remain (i.e., newly created columns are all-NaN)
placeholder_cols = [c for c in target_cols if df_out[c].isna().all()]
if placeholder_cols:
    print("\nColumns that are placeholders (all NaN):")
    for c in placeholder_cols:
        print("  ", c)
else:
    print("\nNo placeholder columns — all target cols have values.")


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from catboost import CatBoostClassifier
import time
import pickle

# -----------------------------------------
# 1️⃣ Load cleaned dataset
# -----------------------------------------
df = pd.read_csv("/kaggle/input/cleaned-ids/ids2018_cleaned_combined_1.csv")

# Ensure "Label" column exists
if "Label" not in df.columns:
    raise ValueError("❌ ERROR: 'Label' column not found in dataset!")

# -----------------------------------------
# 2️⃣ Split features and target
# -----------------------------------------
X = df.drop("Label", axis=1)
y = df["Label"]

# -----------------------------------------
# 3️⃣ Train-test split (80-20)
# -----------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

# -----------------------------------------
# 4️⃣ Train CatBoost model + record time
# -----------------------------------------
model = CatBoostClassifier(
    iterations=300,
    learning_rate=0.1,
    depth=8,
    loss_function='MultiClass',
    verbose=False
)

start_time = time.time()
model.fit(X_train, y_train)
end_time = time.time()
training_time = end_time - start_time

# -----------------------------------------
# 5️⃣ Predictions
# -----------------------------------------
y_pred = model.predict(X_test)
y_pred = y_pred.flatten()

# -----------------------------------------
# 6️⃣ Metrics
# -----------------------------------------
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average="weighted")
recall = recall_score(y_test, y_pred, average="weighted")
precision = precision_score(y_test, y_pred, average="weighted")

# -----------------------------------------
# 7️⃣ Save model to Pickle
# -----------------------------------------
pickle_path = "/kaggle/working/catboost_ids2018_model.pkl"
with open(pickle_path, "wb") as f:
    pickle.dump(model, f)

# -----------------------------------------
# 8️⃣ Print results
# -----------------------------------------
print("✔️ CatBoost Training Complete\n")
print("📁 Model saved as:", pickle_path)
print("📌 Accuracy:", acc)
print("📌 F1 Score (weighted):", f1)
print("📌 Recall (weighted):", recall)
print("📌 Precision (weighted):", precision)
print(f"⏱️ Time Taken to Train: {training_time:.2f} seconds")


In [ ]:
# Code cell: Evaluate CatBoost model and print metrics
import time
import os
import pickle
import joblib
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, confusion_matrix

MODEL_PATH = "/kaggle/working/catboost_ids2018_model.pkl"
TEST_CSV  = "/kaggle/working/2019_cleaned_combined_binary.csv"
OUT_CSV   = "/kaggle/working/2019_with_preds.csv"

# ---------- 1) Load test data ----------
if not os.path.exists(TEST_CSV):
    raise FileNotFoundError(f"Test CSV not found: {TEST_CSV}")

df_test = pd.read_csv(TEST_CSV)
if 'Label' not in df_test.columns and 'label' in df_test.columns:
    df_test.rename(columns={'label': 'Label'}, inplace=True)

if 'Label' not in df_test.columns:
    raise ValueError("Test CSV must contain a 'Label' column (0/1 or 'Benign'/'Bot').")

# Normalize label to binary 0/1 if necessary
def normalize_label_series(s):
    s2 = s.astype(str).str.strip().str.lower()
    # handle typical variants
    benign_keywords = {"benign","begnin","bengin","normal","good"}
    return s2.apply(lambda x: 0 if x in benign_keywords or "benign" in x else (1 if x not in ["0","0.0","0.00","false","f","no","n"] else 0)).astype(int)

try:
    # if labels already 0/1 ints, this keeps them
    if set(df_test['Label'].unique()) <= {0,1}:
        y_true = df_test['Label'].astype(int).to_numpy()
    else:
        y_true = normalize_label_series(df_test['Label']).to_numpy()
except Exception:
    # fallback robust conversion
    y_true = normalize_label_series(df_test['Label']).to_numpy()

# Prepare feature frame (drop Label)
X_test = df_test.drop(columns=['Label']).copy()

# ---------- 2) Load model (try joblib then pickle) ----------
if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"Model file not found: {MODEL_PATH}")

t0 = time.time()
model = None
load_err = None
for loader in (joblib.load, pickle.load):
    try:
        # joblib.load expects a path; pickle.load expects file handle
        if loader is joblib.load:
            model = loader(MODEL_PATH)
        else:
            with open(MODEL_PATH, 'rb') as fh:
                model = loader(fh)
        break
    except Exception as e:
        load_err = e
        model = None

if model is None:
    raise RuntimeError(f"Failed to load model from {MODEL_PATH}. Last error: {load_err}")

# ---------- 3) Align test columns to model's expected features ----------
# Try to get model feature names (CatBoost stores them as attribute feature_names_)
model_feature_names = None
if hasattr(model, 'feature_names_'):
    try:
        model_feature_names = list(model.feature_names_)
    except Exception:
        model_feature_names = None

# If model does not expose feature names, try attribute 'feature_names' or 'get_feature_names'
if model_feature_names is None:
    if hasattr(model, 'feature_names'):
        try:
            model_feature_names = list(model.feature_names)
        except Exception:
            model_feature_names = None

# If still None, fallback to using X_test columns as-is (order preserved)
if model_feature_names is None:
    # Use the test columns order
    model_feature_names = X_test.columns.tolist()

# Now ensure X_test has all required model features in the right order:
missing_cols = [c for c in model_feature_names if c not in X_test.columns]
extra_cols = [c for c in X_test.columns if c not in model_feature_names]

# Add missing cols as zeros (or NaN -> replaced with 0)
if missing_cols:
    for c in missing_cols:
        # add numeric zero column
        X_test[c] = 0

# Drop extras
if extra_cols:
    X_test = X_test.drop(columns=extra_cols)

# Reorder to model_feature_names
X_test = X_test[model_feature_names]

# Convert dtype numeric where possible
for col in X_test.columns:
    # attempt to coerce to numeric; if fails, leave as-is (CatBoost may accept strings/categorical)
    try:
        X_test[col] = pd.to_numeric(X_test[col], errors='ignore')
    except Exception:
        pass

# ---------- 4) Predict ----------
# time prediction (and include load time)
pred_start = time.time()
# Many CatBoost models implement predict() returning class labels; some may require Pool
try:
    y_pred = model.predict(X_test)
except Exception:
    # try using model.predict with Pool if CatBoost installed
    try:
        from catboost import Pool
        y_pred = model.predict(Pool(X_test))
    except Exception as e:
        raise RuntimeError(f"Prediction failed: {e}")

# try predict_proba if available
y_proba = None
try:
    y_proba = model.predict_proba(X_test)
except Exception:
    try:
        from catboost import Pool
        y_proba = model.predict_proba(Pool(X_test))
    except Exception:
        y_proba = None

pred_end = time.time()
total_time = pred_end - t0  # load + predict time
predict_time = pred_end - pred_start

# Ensure predictions are 0/1 ints
y_pred_arr = np.array(y_pred).squeeze()
# Many models output floats; convert threshold if necessary
if y_pred_arr.dtype.kind in 'f' and set(np.unique(y_pred_arr)).issubset({0.0, 1.0}) is False:
    # round to nearest int
    y_pred_labels = (np.rint(y_pred_arr)).astype(int)
else:
    y_pred_labels = y_pred_arr.astype(int)

# ---------- 5) Metrics ----------
acc = accuracy_score(y_true, y_pred_labels)
f1_w = f1_score(y_true, y_pred_labels, average='weighted', zero_division=0)
rec_w = recall_score(y_true, y_pred_labels, average='weighted', zero_division=0)
prec_w = precision_score(y_true, y_pred_labels, average='weighted', zero_division=0)

# ---------- 6) Save predictions appended to test CSV ----------
out_df = df_test.copy()
out_df['pred_label'] = y_pred_labels
if y_proba is not None:
    # if binary, take probability of positive class (col index 1)
    try:
        if y_proba.shape[1] == 2:
            out_df['pred_proba_pos'] = np.array(y_proba)[:, 1]
        else:
            # for multi-class, store entire array as string
            out_df['pred_proba'] = [list(map(float, row)) for row in np.array(y_proba)]
    except Exception:
        pass

out_df.to_csv(OUT_CSV, index=False)

# ---------- 7) Print nicely formatted results ----------
print(f"Accuracy: {acc:.12f}")
print(f"📌 F1 Score (weighted): {f1_w:.12f}")
print(f"📌 Recall (weighted): {rec_w:.12f}")
print(f"📌 Precision (weighted): {prec_w:.12f}")
print(f"⏱️ Time Taken (load + predict): {total_time:.2f} seconds")
print(f"⏱️ Time Taken (predict only): {predict_time:.4f} seconds")

print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred_labels))

print("\nClassification report (per-class):")
print(classification_report(y_true, y_pred_labels, digits=6))

print(f"\nSaved predictions CSV: {OUT_CSV}")


In [ ]:
import joblib, pickle, os, pandas as pd, numpy as np, time
from pathlib import Path

MODEL_PATH = "/kaggle/working/catboost_ids2018_model.pkl"
TEST_CSV  = "/kaggle/working/2019_cleaned_combined_binary.csv"

model = None
for loader in (joblib.load, pickle.load):
    try:
        model = loader(MODEL_PATH) if loader is joblib.load else loader(open(MODEL_PATH,'rb'))
        break
    except Exception:
        model = None
if model is None:
    raise RuntimeError("Failed to load model.")

df_test = pd.read_csv(TEST_CSV)
if 'Label' in df_test.columns:
    X_test = df_test.drop(columns=['Label']).copy()
else:
    X_test = df_test.copy()

# get model features
model_feature_names = None
if hasattr(model, 'feature_names_'):
    model_feature_names = list(model.feature_names_)
elif hasattr(model, 'feature_names'):
    model_feature_names = list(model.feature_names)
else:
    model_feature_names = None

print("Model exposes feature names?:", model_feature_names is not None)
if model_feature_names is not None:
    print("Number of model features:", len(model_feature_names))
    missing_cols = [c for c in model_feature_names if c not in X_test.columns]
    extra_cols = [c for c in X_test.columns if c not in model_feature_names]
    print("Missing columns (in test but required by model):", len(missing_cols))
    print("Example missing (first 20):", missing_cols[:20])
    print("Extra columns in test not used by model:", len(extra_cols))
    print("Example extras (first 20):", extra_cols[:20])
else:
    print("No model feature list available — model likely relies on positional input. You must ensure test columns order matches training order exactly.")


In [ ]:
dff = pd.read_csv("/kaggle/input/cleaned-ids/ids2018_cleaned_combined_1.csv")
dff.columns
df.shape

In [ ]:
dfff = pd.read_parquet("/kaggle/input/cicids2017/Botnet-Friday-no-metadata.parquet")
dfff.columns

In [ ]:
# Code cell: rename / drop / add / reorder and save
import pandas as pd
import numpy as np
from pathlib import Path

# ---------- EDIT PATHS IF NEEDED ----------
input_path = "/kaggle/input/cicids2017/Botnet-Friday-no-metadata.parquet"   # your current file (adjust if different)
output_path = "/kaggle/working/inital_2017_catboost.csv"
Path(output_path).parent.mkdir(parents=True, exist_ok=True)

# ---------- Target schema (exact order you requested) ----------
target_cols = [
    'Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts',
    'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
    'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
    'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
    'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
    'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
    'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
    'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
    'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd URG Flags', 'Fwd Header Len',
    'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min',
    'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var',
    'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt',
    'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt',
    'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg',
    'Subflow Fwd Pkts', 'Subflow Fwd Byts', 'Subflow Bwd Pkts',
    'Subflow Bwd Byts', 'Init Fwd Win Byts', 'Init Bwd Win Byts',
    'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
    'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max',
    'Idle Min', 'Label', 'Flow ID', 'Src IP', 'Src Port', 'Dst IP'
]

# ---------- Mapping: source_column_name -> target_column_name ----------
# Built from the source columns you supplied earlier.
rename_map = {
    # direct / straightforward renames
    'Protocol': 'Protocol',
    'Flow Duration': 'Flow Duration',
    'Total Fwd Packets': 'Tot Fwd Pkts',
    'Total Backward Packets': 'Tot Bwd Pkts',
    'Fwd Packets Length Total': 'TotLen Fwd Pkts',
    'Bwd Packets Length Total': 'TotLen Bwd Pkts',
    'Fwd Packet Length Max': 'Fwd Pkt Len Max',
    'Fwd Packet Length Min': 'Fwd Pkt Len Min',
    'Fwd Packet Length Mean': 'Fwd Pkt Len Mean',
    'Fwd Packet Length Std': 'Fwd Pkt Len Std',
    'Bwd Packet Length Max': 'Bwd Pkt Len Max',
    'Bwd Packet Length Min': 'Bwd Pkt Len Min',
    'Bwd Packet Length Mean': 'Bwd Pkt Len Mean',
    'Bwd Packet Length Std': 'Bwd Pkt Len Std',
    'Flow Bytes/s': 'Flow Byts/s',
    'Flow Packets/s': 'Flow Pkts/s',
    'Flow IAT Mean': 'Flow IAT Mean',
    'Flow IAT Std': 'Flow IAT Std',
    'Flow IAT Max': 'Flow IAT Max',
    'Flow IAT Min': 'Flow IAT Min',
    'Fwd IAT Total': 'Fwd IAT Tot',
    'Fwd IAT Mean': 'Fwd IAT Mean',
    'Fwd IAT Std': 'Fwd IAT Std',
    'Fwd IAT Max': 'Fwd IAT Max',
    'Fwd IAT Min': 'Fwd IAT Min',
    'Bwd IAT Total': 'Bwd IAT Tot',
    'Bwd IAT Mean': 'Bwd IAT Mean',
    'Bwd IAT Std': 'Bwd IAT Std',
    'Bwd IAT Max': 'Bwd IAT Max',
    'Bwd IAT Min': 'Bwd IAT Min',
    'Fwd PSH Flags': 'Fwd PSH Flags',
    # 'Bwd PSH Flags' - source only; will be dropped (target doesn't include)
    'Fwd URG Flags': 'Fwd URG Flags',
    # 'Bwd URG Flags' - source only; drop
    'Fwd Header Length': 'Fwd Header Len',
    'Bwd Header Length': 'Bwd Header Len',
    'Fwd Packets/s': 'Fwd Pkts/s',
    'Bwd Packets/s': 'Bwd Pkts/s',
    'Packet Length Min': 'Pkt Len Min',
    'Packet Length Max': 'Pkt Len Max',
    'Packet Length Mean': 'Pkt Len Mean',
    'Packet Length Std': 'Pkt Len Std',
    'Packet Length Variance': 'Pkt Len Var',
    'FIN Flag Count': 'FIN Flag Cnt',
    'SYN Flag Count': 'SYN Flag Cnt',
    'RST Flag Count': 'RST Flag Cnt',
    'PSH Flag Count': 'PSH Flag Cnt',
    'ACK Flag Count': 'ACK Flag Cnt',
    'URG Flag Count': 'URG Flag Cnt',
    'CWE Flag Count': 'CWE Flag Count',
    'ECE Flag Count': 'ECE Flag Cnt',
    'Down/Up Ratio': 'Down/Up Ratio',
    'Avg Packet Size': 'Pkt Size Avg',
    'Avg Fwd Segment Size': 'Fwd Seg Size Avg',
    'Avg Bwd Segment Size': 'Bwd Seg Size Avg',
    # Bulk / Avg Bulk Rate fields -> many target schema doesn't have these; we will drop them
    # 'Fwd Avg Bytes/Bulk': (drop),
    # 'Fwd Avg Packets/Bulk': (drop),
    # 'Fwd Avg Bulk Rate': (drop),
    # 'Bwd Avg Bytes/Bulk': (drop),
    # 'Bwd Avg Packets/Bulk': (drop),
    # 'Bwd Avg Bulk Rate': (drop),
    'Subflow Fwd Packets': 'Subflow Fwd Pkts',
    'Subflow Fwd Bytes': 'Subflow Fwd Byts',
    'Subflow Bwd Packets': 'Subflow Bwd Pkts',
    'Subflow Bwd Bytes': 'Subflow Bwd Byts',
    'Init Fwd Win Bytes': 'Init Fwd Win Byts',
    'Init Bwd Win Bytes': 'Init Bwd Win Byts',
    'Fwd Act Data Packets': 'Fwd Act Data Pkts',
    'Fwd Seg Size Min': 'Fwd Seg Size Min',
    'Active Mean': 'Active Mean',
    'Active Std': 'Active Std',
    'Active Max': 'Active Max',
    'Active Min': 'Active Min',
    'Idle Mean': 'Idle Mean',
    'Idle Std': 'Idle Std',
    'Idle Max': 'Idle Max',
    'Idle Min': 'Idle Min',
    'Label': 'Label'
}

# ---------- Load input ----------
print("Loading:", input_path)
df = pd.read_parquet(input_path)
print("Input shape:", df.shape)
print("Input columns count:", len(df.columns))

# ---------- Rename columns where mapping exists ----------
# Build inverse map (source -> target) from rename_map already is source->target
existing_source_cols = [c for c in rename_map.keys() if c in df.columns]
rename_dict = {src: rename_map[src] for src in existing_source_cols}
df = df.rename(columns=rename_dict)
print(f"Renamed {len(rename_dict)} columns.")

# ---------- Drop columns that are neither renamed into target nor present in target ----------
# Keep any column that is already in target_cols or was renamed into target.
cols_to_keep = set(target_cols)  # desired final columns
# also keep any target-like 'Label' etc.
current_cols = set(df.columns)

# Columns to drop: those present in current df but not in target columns
drop_candidates = [c for c in df.columns if c not in cols_to_keep]
if drop_candidates:
    print(f"Dropping {len(drop_candidates)} source-only columns (not in target):")
    print(drop_candidates[:50])  # preview
    df = df.drop(columns=drop_candidates)

# ---------- Add missing target columns as NaN placeholders ----------
missing_targets = [c for c in target_cols if c not in df.columns]
if missing_targets:
    print(f"Adding {len(missing_targets)} missing target columns as NaN placeholders:")
    print(missing_targets)
    for c in missing_targets:
        df[c] = np.nan

# ---------- Reorder columns exactly to target_cols ----------
df = df[target_cols].copy()

# ---------- Final checks ----------
print("Final shape:", df.shape)
print("Final columns (first 120 chars):")
print(df.columns.tolist())

# ---------- Save ----------
df.to_csv(output_path, index=False)
print("Saved converted file to:", output_path)


In [34]:
import pandas as pd

path = "/kaggle/working/inital_2017_catboost.csv"

df = pd.read_csv(path)

# Show unique values and their counts
print("Unique values in Label column:")
print(df['Label'].value_counts(dropna=False))

# Also show raw unique values
print("\nRaw unique values:")
print(df['Label'].unique())


Unique values in Label column:
Label
Benign    174601
Bot         1437
Name: count, dtype: int64

Raw unique values:
['Benign' 'Bot']


In [33]:
import pandas as pd

path = "/kaggle/working/inital_2017_catboost.csv"
df = pd.read_csv(path)

# Normalize case
df['Label'] = df['Label'].astype(str).str.strip().str.lower()

# Map: benign → 0, bot → 1
df['Label'] = df['Label'].apply(lambda x: 0 if x == "benign" else 1)

# Save back
df.to_csv("/kaggle/working/inital_2017_catboost_binary.csv", index=False)

print("Label conversion complete!")
print(df['Label'].value_counts())


Label conversion complete!
Label
0    174601
1      1437
Name: count, dtype: int64


In [36]:
gf=pd.read_csv("/kaggle/working/inital_2017_catboost_binary.csv")
# Show unique values and their counts
print("Unique values in Label column:")
print(gf['Label'].value_counts(dropna=False))

# Also show raw unique values
print("\nRaw unique values:")
print(gf['Label'].unique())


Unique values in Label column:
Label
0    174601
1      1437
Name: count, dtype: int64

Raw unique values:
[0 1]


In [37]:
import pandas as pd
import numpy as np
import joblib, pickle, os, time
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix, classification_report
from catboost import Pool

MODEL_PATH = "/kaggle/working/catboost_ids2018_model.pkl"
TEST_PATH  = "/kaggle/working/inital_2017_catboost_binary.csv"

# ------------------ 1. Load test data ------------------
df = pd.read_csv(TEST_PATH)

if 'Label' not in df.columns:
    raise ValueError("❌ 'Label' column not found in test CSV")

y_true = df['Label'].astype(int).values
X_test = df.drop(columns=['Label']).copy()

# ------------------ 2. Load model ------------------
model = None
for loader in (joblib.load, pickle.load):
    try:
        if loader is joblib.load:
            model = loader(MODEL_PATH)
        else:
            with open(MODEL_PATH, 'rb') as f:
                model = loader(f)
        break
    except:
        pass

if model is None:
    raise RuntimeError("❌ Could not load model.")

# ------------------ 3. Align test columns to model features ------------------
model_features = None

if hasattr(model, "feature_names_"):
    model_features = list(model.feature_names_)
elif hasattr(model, "feature_names"):
    model_features = list(model.feature_names)

# If model exposes feature names → STRICT MATCHING
if model_features is not None:
    missing = [c for c in model_features if c not in X_test.columns]
    extra   = [c for c in X_test.columns if c not in model_features]

    # Add missing columns as 0
    for c in missing:
        X_test[c] = 0

    # Drop extras
    if extra:
        X_test = X_test.drop(columns=extra)

    # Reorder to match model
    X_test = X_test[model_features]

else:
    print("⚠ Model does NOT expose feature names. Using test columns as-is.")

# Convert to numeric where possible
for c in X_test.columns:
    try:
        X_test[c] = pd.to_numeric(X_test[c], errors="coerce").fillna(0)
    except:
        pass

# ------------------ 4. Predict ------------------
start = time.time()

try:
    y_pred = model.predict(X_test)
except:
    y_pred = model.predict(Pool(X_test))

y_pred = np.array(y_pred).astype(int)

end = time.time()

# ------------------ 5. Metrics ------------------
acc  = accuracy_score(y_true, y_pred)
f1   = f1_score(y_true, y_pred, average='weighted', zero_division=0)
rec  = recall_score(y_true, y_pred, average='weighted', zero_division=0)
prec = precision_score(y_true, y_pred, average='weighted', zero_division=0)

# ------------------ 6. Print results ------------------
print(f"Accuracy: {acc:.12f}")
print(f"📌 F1 Score (weighted): {f1:.12f}")
print(f"📌 Recall (weighted): {rec:.12f}")
print(f"📌 Precision (weighted): {prec:.12f}")
print(f"⏱️ Time Taken: {end-start:.2f} seconds\n")

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

print("\nClassification Report:")
print(classification_report(y_true, y_pred, digits=6))


Accuracy: 0.008163010259
📌 F1 Score (weighted): 0.000132190401
📌 Recall (weighted): 0.008163010259
📌 Precision (weighted): 0.000066634736
⏱️ Time Taken: 0.22 seconds

Confusion Matrix:
[[     0 174601]
 [     0   1437]]

Classification Report:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0   0.000000  0.000000  0.000000    174601
           1   0.008163  1.000000  0.016194      1437

    accuracy                       0.008163    176038
   macro avg   0.004082  0.500000  0.008097    176038
weighted avg   0.000067  0.008163  0.000132    176038



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [38]:
import joblib, pickle

MODEL_PATH = "/kaggle/working/catboost_ids2018_model.pkl"

model = None
for loader in (joblib.load, pickle.load):
    try:
        if loader is joblib.load:
            model = loader(MODEL_PATH)
        else:
            with open(MODEL_PATH, "rb") as f:
                model = loader(f)
        break
    except:
        pass

# Print model feature names
print("Model feature names:")
if hasattr(model, "feature_names_"):
    print(model.feature_names_)
elif hasattr(model, "feature_names"):
    print(model.feature_names)
else:
    print("❌ Model does NOT expose feature names. Need training code.")


Model feature names:
['Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Subflow Fwd Pkts', 'Subflow Fwd Byts', 'Subflow Bwd Pkts', 'Subflow Bwd Byt

In [39]:
df.columns

Index(['Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts',
       'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd URG Flags', 'Fwd Header Len',
       'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min',
       'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var',
       'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt',
       'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt',
       'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg',


In [40]:
# Code cell: Align & predict (copy->paste into Kaggle)
import os, time, pickle, joblib
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix, classification_report

MODEL_PATH = "/kaggle/working/catboost_ids2018_model.pkl"
TEST_PATH  = "/kaggle/working/inital_2017_catboost_binary.csv"
OUT_PATH   = "/kaggle/working/inital_2017_catboost_with_preds.csv"
Path(OUT_PATH).parent.mkdir(parents=True, exist_ok=True)

# 1) Load test file and show columns
df = pd.read_csv(TEST_PATH)
print("Test file loaded. Shape:", df.shape)
print("\nTest file columns:")
print(df.columns.tolist())

# 2) Ensure Label binary
if 'Label' not in df.columns:
    raise ValueError("Label column missing in test file.")
df['Label'] = df['Label'].astype(str).str.strip().str.lower().apply(lambda x: 0 if "benign" in x else 1).astype(int)
y_true = df['Label'].to_numpy()

# 3) Load model
model = None
last_err = None
for loader in (joblib.load, pickle.load):
    try:
        model = loader(MODEL_PATH) if loader is joblib.load else loader(open(MODEL_PATH,'rb'))
        break
    except Exception as e:
        last_err = e
if model is None:
    raise RuntimeError(f"Failed to load model. Last error: {last_err}")

# 4) Get model's expected feature list
if hasattr(model, "feature_names_"):
    model_features = list(model.feature_names_)
elif hasattr(model, "feature_names"):
    model_features = list(model.feature_names)
else:
    raise RuntimeError("Model does not expose feature names. Provide training feature list.")

print("\nModel expects", len(model_features), "features.")
print("First 10 model features:", model_features[:10])

# 5) Rename known variants if present (safe no-op if not present)
# (If you've already renamed to the model names, this block will do nothing.)
rename_map = {
    'Total Fwd Packets': 'Tot Fwd Pkts',
    'Total Backward Packets': 'Tot Bwd Pkts',
    'Fwd Packets Length Total': 'TotLen Fwd Pkts',
    'Bwd Packets Length Total': 'TotLen Bwd Pkts',
    'Fwd Packet Length Max': 'Fwd Pkt Len Max',
    'Fwd Packet Length Min': 'Fwd Pkt Len Min',
    'Fwd Packet Length Mean': 'Fwd Pkt Len Mean',
    'Fwd Packet Length Std': 'Fwd Pkt Len Std',
    'Bwd Packet Length Max': 'Bwd Pkt Len Max',
    'Bwd Packet Length Min': 'Bwd Pkt Len Min',
    'Bwd Packet Length Mean': 'Bwd Pkt Len Mean',
    'Bwd Packet Length Std': 'Bwd Pkt Len Std',
    'Flow Bytes/s': 'Flow Byts/s',
    'Flow Packets/s': 'Flow Pkts/s',
    'Packet Length Min': 'Pkt Len Min',
    'Packet Length Max': 'Pkt Len Max',
    'Packet Length Mean': 'Pkt Len Mean',
    'Packet Length Std': 'Pkt Len Std',
    'Packet Length Variance': 'Pkt Len Var',
    'FIN Flag Count': 'FIN Flag Cnt',
    'SYN Flag Count': 'SYN Flag Cnt',
    'RST Flag Count': 'RST Flag Cnt',
    'PSH Flag Count': 'PSH Flag Cnt',
    'ACK Flag Count': 'ACK Flag Cnt',
    'URG Flag Count': 'URG Flag Cnt',
    'CWE Flag Count': 'CWE Flag Count',
    'ECE Flag Count': 'ECE Flag Cnt',
    'Avg Packet Size': 'Pkt Size Avg',
    'Avg Fwd Segment Size': 'Fwd Seg Size Avg',
    'Avg Bwd Segment Size': 'Bwd Seg Size Avg',
    'Subflow Fwd Packets': 'Subflow Fwd Pkts',
    'Subflow Fwd Bytes': 'Subflow Fwd Byts',
    'Subflow Bwd Packets': 'Subflow Bwd Pkts',
    'Subflow Bwd Bytes': 'Subflow Bwd Byts',
    'Init Fwd Win Bytes': 'Init Fwd Win Byts',
    'Init Bwd Win Bytes': 'Init Bwd Win Byts',
    'Fwd Act Data Packets': 'Fwd Act Data Pkts',
    'Fwd Seg Size Min': 'Fwd Seg Size Min',
    'Fwd Header Length': 'Fwd Header Len',
    'Bwd Header Length': 'Bwd Header Len',
    'Fwd Packets/s': 'Fwd Pkts/s',
    'Bwd Packets/s': 'Bwd Pkts/s',
    'Flow Bytes/s': 'Flow Byts/s',
    'Flow Packets/s': 'Flow Pkts/s',
    'Packet Length Variance': 'Pkt Len Var'
}
present_renames = {k:v for k,v in rename_map.items() if k in df.columns}
if present_renames:
    df = df.rename(columns=present_renames)
    print(f"\nApplied {len(present_renames)} renames to match model naming.")

# 6) Add missing model features with sensible defaults, drop extras
missing = [c for c in model_features if c not in df.columns]
extra   = [c for c in df.columns if c not in model_features and c != 'Label']

print("\nMissing features (will be added):", missing)
print("Extra columns (will be dropped):", extra[:50])

for c in missing:
    if c in ('Src IP','Dst IP','Flow ID'):
        df[c] = ""   # string placeholder
    elif c in ('Src Port','Dst Port','Timestamp'):
        df[c] = 0    # numeric placeholder
    else:
        df[c] = 0

if extra:
    df = df.drop(columns=extra)

# 7) Reorder exactly to model_features
X = df[model_features].copy()
print("\nPrepared X shape:", X.shape)

# 8) Convert numeric-like columns to numeric (keep IPs/FlowID as strings)
non_numeric_keep = {'Src IP','Dst IP','Flow ID'}
for c in X.columns:
    if c in non_numeric_keep:
        X[c] = X[c].astype(str).fillna("")
    else:
        X[c] = pd.to_numeric(X[c], errors='coerce').fillna(0)

# 9) Quick sanity: show first 5 rows of prepared features
print("\nFirst 5 rows of prepared features:")
display(X.head())

# 10) Predict and evaluate
t0 = time.time()
try:
    y_pred = model.predict(X)
except Exception:
    from catboost import Pool
    y_pred = model.predict(Pool(X))
t1 = time.time()

y_pred = np.array(y_pred).squeeze()
if y_pred.dtype.kind == 'f' and not set(np.unique(y_pred)).issubset({0.0,1.0}):
    y_pred = np.rint(y_pred).astype(int)
else:
    y_pred = y_pred.astype(int)

acc  = accuracy_score(y_true, y_pred)
f1_w = f1_score(y_true, y_pred, average='weighted', zero_division=0)
rec_w = recall_score(y_true, y_pred, average='weighted', zero_division=0)
prec_w = precision_score(y_true, y_pred, average='weighted', zero_division=0)

print(f"\nAccuracy: {acc:.12f}")
print(f"📌 F1 Score (weighted): {f1_w:.12f}")
print(f"📌 Recall (weighted): {rec_w:.12f}")
print(f"📌 Precision (weighted): {prec_w:.12f}")
print(f"⏱️ Time Taken (predict only): {(t1-t0):.4f} seconds\n")

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

print("\nClassification Report:")
print(classification_report(y_true, y_pred, digits=6))

# 11) Save predictions appended
df_out = df.copy()
df_out['pred_label'] = y_pred

# try predict_proba
try:
    y_proba = model.predict_proba(X)
    y_proba = np.array(y_proba)
    if y_proba.ndim == 2 and y_proba.shape[1] == 2:
        df_out['pred_proba_pos'] = y_proba[:,1]
    else:
        df_out['pred_proba'] = [list(map(float,row)) for row in y_proba]
except Exception:
    pass

df_out.to_csv(OUT_PATH, index=False)
print(f"\nSaved predictions to: {OUT_PATH}")


Test file loaded. Shape: (176038, 76)

Test file columns:
['Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Subflow Fwd Pkts', 'Subflow Fwd Byts',

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Flow ID,Src IP,Src Port,Dst IP
0,0.0,6,0.0,112740690,32,16,6448,1152,403,0,...,380,343,16100000.0,498804.80,16400000,15400000,nan,nan,0.0,nan
1,0.0,6,0.0,112740560,32,16,6448,5056,403,0,...,330,285,16100000.0,498793.66,16400000,15400000,nan,nan,0.0,nan
2,0.0,0,0.0,113757377,545,0,0,0,0,0,...,18900000,19,12200000.0,6935824.00,20800000,5504997,nan,nan,0.0,nan
3,0.0,17,0.0,100126,22,0,616,0,28,28,...,0,0,0.0,0.00,0,0,nan,nan,0.0,nan
4,0.0,0,0.0,54760,4,0,0,0,0,0,...,0,0,0.0,0.00,0,0,nan,nan,0.0,nan



Accuracy: 1.000000000000
📌 F1 Score (weighted): 1.000000000000
📌 Recall (weighted): 1.000000000000
📌 Precision (weighted): 1.000000000000
⏱️ Time Taken (predict only): 0.2692 seconds

Confusion Matrix:
[[176038]]

Classification Report:
              precision    recall  f1-score   support

           1   1.000000  1.000000  1.000000    176038

    accuracy                       1.000000    176038
   macro avg   1.000000  1.000000  1.000000    176038
weighted avg   1.000000  1.000000  1.000000    176038


Saved predictions to: /kaggle/working/inital_2017_catboost_with_preds.csv


In [71]:
a=pd.read_csv("/kaggle/working/processed_botnet_2017.csv")
a.columns

Index(['Dst Port', 'Timestamp', 'Flow Duration', 'Fwd Pkt Len Min',
       'Fwd Pkt Len Mean', 'Bwd Pkt Len Max', 'Bwd Pkt Len Min',
       'Bwd Pkt Len Std', 'Flow Pkts/s', 'Fwd IAT Std', 'Fwd Header Len',
       'Fwd Pkts/s', 'Pkt Len Mean', 'Pkt Len Var', 'SYN Flag Cnt',
       'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Subflow Bwd Byts',
       'Init Bwd Win Byts', 'Fwd Seg Size Min', 'Idle Mean', 'Idle Std',
       'Idle Min', 'Src IP', 'Dst IP', 'Label'],
      dtype='object')

In [72]:
import joblib, pickle

MODEL_PATH = "/kaggle/working/catboost_ids2018_model.pkl"

# Try loading using joblib first, then pickle
model = None
load_err = None

for loader in (joblib.load, pickle.load):
    try:
        if loader is joblib.load:
            model = loader(MODEL_PATH)
        else:
            with open(MODEL_PATH, "rb") as f:
                model = loader(f)
        break
    except Exception as e:
        load_err = e

if model is None:
    raise RuntimeError(f"❌ Failed to load model: {load_err}")

# Print feature names used by the model
if hasattr(model, "feature_names_"):
    features = list(model.feature_names_)
elif hasattr(model, "feature_names"):
    features = list(model.feature_names)
else:
    raise RuntimeError("❌ Model does not contain feature names.")

print("Total features:", len(features))
print("\nFeature list in correct order:\n")
for idx, f in enumerate(features):
    print(f"{idx+1:02d}. {f}")


Total features: 75

Feature list in correct order:

01. Dst Port
02. Protocol
03. Timestamp
04. Flow Duration
05. Tot Fwd Pkts
06. Tot Bwd Pkts
07. TotLen Fwd Pkts
08. TotLen Bwd Pkts
09. Fwd Pkt Len Max
10. Fwd Pkt Len Min
11. Fwd Pkt Len Mean
12. Fwd Pkt Len Std
13. Bwd Pkt Len Max
14. Bwd Pkt Len Min
15. Bwd Pkt Len Mean
16. Bwd Pkt Len Std
17. Flow Byts/s
18. Flow Pkts/s
19. Flow IAT Mean
20. Flow IAT Std
21. Flow IAT Max
22. Flow IAT Min
23. Fwd IAT Tot
24. Fwd IAT Mean
25. Fwd IAT Std
26. Fwd IAT Max
27. Fwd IAT Min
28. Bwd IAT Tot
29. Bwd IAT Mean
30. Bwd IAT Std
31. Bwd IAT Max
32. Bwd IAT Min
33. Fwd PSH Flags
34. Fwd URG Flags
35. Fwd Header Len
36. Bwd Header Len
37. Fwd Pkts/s
38. Bwd Pkts/s
39. Pkt Len Min
40. Pkt Len Max
41. Pkt Len Mean
42. Pkt Len Std
43. Pkt Len Var
44. FIN Flag Cnt
45. SYN Flag Cnt
46. RST Flag Cnt
47. PSH Flag Cnt
48. ACK Flag Cnt
49. URG Flag Cnt
50. CWE Flag Count
51. ECE Flag Cnt
52. Down/Up Ratio
53. Pkt Size Avg
54. Fwd Seg Size Avg
55. Bwd Seg 

In [73]:
gg=pd.read_parquet("/kaggle/input/cicids2017/Botnet-Friday-no-metadata.parquet")
gg.columns

Index(['Protocol', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Fwd Packets Length Total',
       'Bwd Packets Length Total', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags',
       'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Count', 'RST Fla

In [74]:
gg.shape

(176038, 78)

In [75]:
# ---------- prepare_and_save_for_model.py ----------
import pandas as pd
import numpy as np
from pathlib import Path

# ---------- paths ----------
src_parquet = "/kaggle/input/cicids2017/Botnet-Friday-no-metadata.parquet"
out_parquet = "/kaggle/working/inital_2017_catboost_ready.parquet"
out_csv     = "/kaggle/working/inital_2017_catboost_ready.csv"
Path("/kaggle/working").mkdir(parents=True, exist_ok=True)

# ---------- model feature list (exact order) ----------
model_features = [
 'Dst Port','Protocol','Timestamp','Flow Duration','Tot Fwd Pkts','Tot Bwd Pkts',
 'TotLen Fwd Pkts','TotLen Bwd Pkts','Fwd Pkt Len Max','Fwd Pkt Len Min',
 'Fwd Pkt Len Mean','Fwd Pkt Len Std','Bwd Pkt Len Max','Bwd Pkt Len Min',
 'Bwd Pkt Len Mean','Bwd Pkt Len Std','Flow Byts/s','Flow Pkts/s','Flow IAT Mean',
 'Flow IAT Std','Flow IAT Max','Flow IAT Min','Fwd IAT Tot','Fwd IAT Mean',
 'Fwd IAT Std','Fwd IAT Max','Fwd IAT Min','Bwd IAT Tot','Bwd IAT Mean',
 'Bwd IAT Std','Bwd IAT Max','Bwd IAT Min','Fwd PSH Flags','Fwd URG Flags',
 'Fwd Header Len','Bwd Header Len','Fwd Pkts/s','Bwd Pkts/s','Pkt Len Min',
 'Pkt Len Max','Pkt Len Mean','Pkt Len Std','Pkt Len Var','FIN Flag Cnt',
 'SYN Flag Cnt','RST Flag Cnt','PSH Flag Cnt','ACK Flag Cnt','URG Flag Cnt',
 'CWE Flag Count','ECE Flag Cnt','Down/Up Ratio','Pkt Size Avg','Fwd Seg Size Avg',
 'Bwd Seg Size Avg','Subflow Fwd Pkts','Subflow Fwd Byts','Subflow Bwd Pkts',
 'Subflow Bwd Byts','Init Fwd Win Byts','Init Bwd Win Byts','Fwd Act Data Pkts',
 'Fwd Seg Size Min','Active Mean','Active Std','Active Max','Active Min',
 'Idle Mean','Idle Std','Idle Max','Idle Min','Flow ID','Src IP','Src Port','Dst IP'
]

# ---------- mapping: source column -> model column ----------
# Based on gg.columns you provided
mapping = {
    'Protocol': 'Protocol',
    'Flow Duration': 'Flow Duration',
    'Total Fwd Packets': 'Tot Fwd Pkts',
    'Total Backward Packets': 'Tot Bwd Pkts',
    'Fwd Packets Length Total': 'TotLen Fwd Pkts',
    'Bwd Packets Length Total': 'TotLen Bwd Pkts',
    'Fwd Packet Length Max': 'Fwd Pkt Len Max',
    'Fwd Packet Length Min': 'Fwd Pkt Len Min',
    'Fwd Packet Length Mean': 'Fwd Pkt Len Mean',
    'Fwd Packet Length Std': 'Fwd Pkt Len Std',
    'Bwd Packet Length Max': 'Bwd Pkt Len Max',
    'Bwd Packet Length Min': 'Bwd Pkt Len Min',
    'Bwd Packet Length Mean': 'Bwd Pkt Len Mean',
    'Bwd Packet Length Std': 'Bwd Pkt Len Std',
    'Flow Bytes/s': 'Flow Byts/s',
    'Flow Packets/s': 'Flow Pkts/s',
    'Flow IAT Mean': 'Flow IAT Mean',
    'Flow IAT Std': 'Flow IAT Std',
    'Flow IAT Max': 'Flow IAT Max',
    'Flow IAT Min': 'Flow IAT Min',
    'Fwd IAT Total': 'Fwd IAT Tot',
    'Fwd IAT Mean': 'Fwd IAT Mean',
    'Fwd IAT Std': 'Fwd IAT Std',
    'Fwd IAT Max': 'Fwd IAT Max',
    'Fwd IAT Min': 'Fwd IAT Min',
    'Bwd IAT Total': 'Bwd IAT Tot',
    'Bwd IAT Mean': 'Bwd IAT Mean',
    'Bwd IAT Std': 'Bwd IAT Std',
    'Bwd IAT Max': 'Bwd IAT Max',
    'Bwd IAT Min': 'Bwd IAT Min',
    'Fwd PSH Flags': 'Fwd PSH Flags',
    # 'Bwd PSH Flags' dropped (no target equivalent)
    'Fwd URG Flags': 'Fwd URG Flags',
    # 'Bwd URG Flags' dropped
    'Fwd Header Length': 'Fwd Header Len',
    'Bwd Header Length': 'Bwd Header Len',
    'Fwd Packets/s': 'Fwd Pkts/s',
    'Bwd Packets/s': 'Bwd Pkts/s',
    'Packet Length Min': 'Pkt Len Min',
    'Packet Length Max': 'Pkt Len Max',
    'Packet Length Mean': 'Pkt Len Mean',
    'Packet Length Std': 'Pkt Len Std',
    'Packet Length Variance': 'Pkt Len Var',
    'FIN Flag Count': 'FIN Flag Cnt',
    'SYN Flag Count': 'SYN Flag Cnt',
    'RST Flag Count': 'RST Flag Cnt',
    'PSH Flag Count': 'PSH Flag Cnt',
    'ACK Flag Count': 'ACK Flag Cnt',
    'URG Flag Count': 'URG Flag Cnt',
    'CWE Flag Count': 'CWE Flag Count',
    'ECE Flag Count': 'ECE Flag Cnt',
    'Down/Up Ratio': 'Down/Up Ratio',
    'Avg Packet Size': 'Pkt Size Avg',
    'Avg Fwd Segment Size': 'Fwd Seg Size Avg',
    'Avg Bwd Segment Size': 'Bwd Seg Size Avg',
    'Subflow Fwd Packets': 'Subflow Fwd Pkts',
    'Subflow Fwd Bytes': 'Subflow Fwd Byts',
    'Subflow Bwd Packets': 'Subflow Bwd Pkts',
    'Subflow Bwd Bytes': 'Subflow Bwd Byts',
    'Init Fwd Win Bytes': 'Init Fwd Win Byts',
    'Init Bwd Win Bytes': 'Init Bwd Win Byts',
    'Fwd Act Data Packets': 'Fwd Act Data Pkts',
    'Fwd Seg Size Min': 'Fwd Seg Size Min',
    'Active Mean': 'Active Mean',
    'Active Std': 'Active Std',
    'Active Max': 'Active Max',
    'Active Min': 'Active Min',
    'Idle Mean': 'Idle Mean',
    'Idle Std': 'Idle Std',
    'Idle Max': 'Idle Max',
    'Idle Min': 'Idle Min',
    'Label': 'Label'
}

# ---------- load source parquet ----------
print("Loading:", src_parquet)
gg = pd.read_parquet(src_parquet)
print("Source shape:", gg.shape)
print("Source columns count:", len(gg.columns))

# ---------- rename mapped columns ----------
rename_pairs = {src: tgt for src, tgt in mapping.items() if src in gg.columns}
gg = gg.rename(columns=rename_pairs)
print(f"Renamed {len(rename_pairs)} columns (source->model names).")

# ---------- drop source-only columns not in mapping or model_features ----------
# Determine columns to keep (mapped to model or already a model column or 'Label')
keep_candidates = set(rename_pairs.values()) | set(model_features) | {'Label'}
# but we only want to keep columns that will map to final model features or Label
cols_to_drop = [c for c in gg.columns if c not in keep_candidates]
if cols_to_drop:
    print(f"Dropping {len(cols_to_drop)} source-only columns (examples):", cols_to_drop[:10])
    gg = gg.drop(columns=cols_to_drop)

# ---------- add missing model features with sensible defaults ----------
missing = [c for c in model_features if c not in gg.columns]
print("Missing model features to add:", missing)
for c in missing:
    if c in ('Src IP','Dst IP','Flow ID'):
        gg[c] = ""   # string placeholder
    elif c in ('Src Port','Dst Port','Timestamp'):
        gg[c] = 0    # numeric placeholder
    else:
        gg[c] = 0

# ---------- Ensure Label exists; if not, add as NaN ----------
if 'Label' not in gg.columns:
    gg['Label'] = np.nan
    print("Label column not found in source — created placeholder Label (NaN).")

# ---------- Reorder columns: model_features in order, then Label at the end ----------
final_cols = model_features + ['Label']
gg = gg[final_cols].copy()
print("Final shape:", gg.shape)
print("Final columns (first 40):", gg.columns.tolist()[:40])
print("Final columns (last 10):", gg.columns.tolist()[-10:])

# ---------- convert numeric-like columns to numeric except IP/FlowID ----------
non_numeric_keep = {'Src IP','Dst IP','Flow ID'}
for c in gg.columns:
    if c in non_numeric_keep or c == 'Label':
        # keep Label as-is (do not coerce here; user may want original string labels)
        continue
    # coerce numeric and fill NaN with 0
    gg[c] = pd.to_numeric(gg[c], errors='coerce').fillna(0)

# ---------- Save outputs ----------
gg.to_parquet(out_parquet, index=False)
gg.to_csv(out_csv, index=False)
print("Saved processed files:")
print("  -", out_parquet)
print("  -", out_csv)

# ---------- quick sanity prints ----------
print("\nSample head (first 5 rows):")
print(gg.head(5).iloc[:, :8])   # show first 8 cols
print("\nLabel value counts (if present):")
print(gg['Label'].value_counts(dropna=False))


Loading: /kaggle/input/cicids2017/Botnet-Friday-no-metadata.parquet
Source shape: (176038, 78)
Source columns count: 78
Renamed 70 columns (source->model names).
Dropping 8 source-only columns (examples): ['Bwd PSH Flags', 'Bwd URG Flags', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate']
Missing model features to add: ['Dst Port', 'Timestamp', 'Flow ID', 'Src IP', 'Src Port', 'Dst IP']
Final shape: (176038, 76)
Final columns (first 40): ['Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT 

In [76]:
import pandas as pd

path = "/kaggle/working/inital_2017_catboost_ready.csv"

# Load file
df = pd.read_csv(path)

# Normalize text form
df['Label'] = df['Label'].astype(str).str.strip().str.lower()

# Convert: benign -> 0, all other labels -> 1
df['Label'] = df['Label'].apply(lambda x: 0 if "benign" in x else 1)

# Save back
df.to_csv("/kaggle/working/inital_2017_catboost_ready_binary.csv", index=False)

print("✔ Label column converted successfully!")
print(df['Label'].value_counts())


✔ Label column converted successfully!
Label
0    174601
1      1437
Name: count, dtype: int64


In [79]:
sa=pd.read_csv("/kaggle/working/inital_2017_catboost_ready_binary.csv")
sa.columns

Index(['Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts',
       'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd URG Flags', 'Fwd Header Len',
       'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min',
       'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var',
       'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt',
       'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt',
       'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg',


In [80]:
sa.shape

(176038, 76)

In [81]:
# === Evaluate CatBoost model on provided CSV ===
import os, time, joblib, pickle
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix, classification_report

MODEL_PATH = "/kaggle/working/catboost_ids2018_model.pkl"
TEST_CSV   = "/kaggle/working/inital_2017_catboost_ready_binary.csv"
OUT_CSV    = "/kaggle/working/inital_2017_catboost_ready_with_preds.csv"
Path(OUT_CSV).parent.mkdir(parents=True, exist_ok=True)

# 1) Load test CSV
if not os.path.exists(TEST_CSV):
    raise FileNotFoundError(f"Test CSV not found: {TEST_CSV}")
sa = pd.read_csv(TEST_CSV)
print("Loaded test file. Shape:", sa.shape)

# 2) Ensure Label exists and create y_true (binary)
if 'Label' not in sa.columns:
    raise ValueError("Test CSV must contain a 'Label' column.")

# Normalize common label variants to binary 0/1
def normalize_label_column(s):
    s2 = s.astype(str).str.strip().str.lower()
    benign_set = {"benign","begnin","normal","0","no","false","n","benign\r"}
    return s2.apply(lambda x: 0 if any(k == x or k in x for k in benign_set) else 1).astype(int)

try:
    # If already numeric 0/1, this keeps them
    if set(sa['Label'].dropna().unique()) <= {0, 1}:
        y_true = sa['Label'].astype(int).to_numpy()
    else:
        y_true = normalize_label_column(sa['Label']).to_numpy()
except Exception:
    y_true = normalize_label_column(sa['Label']).to_numpy()

# 3) Load model (joblib then pickle)
model = None
last_err = None
for loader in (joblib.load, pickle.load):
    try:
        model = loader(MODEL_PATH) if loader is joblib.load else loader(open(MODEL_PATH, 'rb'))
        break
    except Exception as e:
        last_err = e
if model is None:
    raise RuntimeError(f"Failed to load model. Last error: {last_err}")

# 4) Get model feature list
if hasattr(model, "feature_names_"):
    model_features = list(model.feature_names_)
elif hasattr(model, "feature_names"):
    model_features = list(model.feature_names)
else:
    raise RuntimeError("Model does not expose feature names. Provide the model feature list.")

print("Model expects", len(model_features), "features.")

# 5) Prepare X_test: drop Label, add missing features, drop extras, reorder
X_df = sa.drop(columns=['Label']).copy()

missing = [c for c in model_features if c not in X_df.columns]
extras  = [c for c in X_df.columns if c not in model_features]

if missing:
    print("Adding missing model features with sensible defaults:", missing)
    for c in missing:
        if c in ('Src IP','Dst IP','Flow ID'):
            X_df[c] = ""      # string placeholders
        elif c in ('Src Port','Dst Port','Timestamp'):
            X_df[c] = 0       # numeric placeholders
        else:
            X_df[c] = 0       # numeric default

if extras:
    print("Dropping extra columns not used by the model (examples):", extras[:10])
    X_df = X_df.drop(columns=extras)

# Reorder exactly
X_df = X_df[model_features].copy()
print("Prepared X_df shape:", X_df.shape)

# 6) Coerce numeric-like columns (preserve IP/FlowID as strings)
non_numeric_keep = {'Src IP','Dst IP','Flow ID'}
for c in X_df.columns:
    if c in non_numeric_keep:
        X_df[c] = X_df[c].astype(str).fillna("")
    else:
        X_df[c] = pd.to_numeric(X_df[c], errors='coerce').fillna(0)

# 7) Predict (time measured)
t0 = time.time()
try:
    y_pred_raw = model.predict(X_df)
except Exception:
    # try CatBoost Pool fallback
    try:
        from catboost import Pool
        y_pred_raw = model.predict(Pool(X_df))
    except Exception as e:
        raise RuntimeError(f"Prediction failed: {e}")
t1 = time.time()

y_pred = np.array(y_pred_raw).squeeze()
# if model returned probabilities or floats, convert to 0/1
if y_pred.dtype.kind == 'f' and not set(np.unique(y_pred)).issubset({0.0,1.0}):
    # rounding is a safe fallback
    y_pred = np.rint(y_pred).astype(int)
else:
    y_pred = y_pred.astype(int)

# 8) Metrics
acc   = accuracy_score(y_true, y_pred)
f1_w  = f1_score(y_true, y_pred, average='weighted', zero_division=0)
rec_w = recall_score(y_true, y_pred, average='weighted', zero_division=0)
prec_w= precision_score(y_true, y_pred, average='weighted', zero_division=0)

# 9) Print results (clean)
print(f"\nAccuracy: {acc:.12f}")
print(f"📌 F1 Score (weighted): {f1_w:.12f}")
print(f"📌 Recall (weighted): {rec_w:.12f}")
print(f"📌 Precision (weighted): {prec_w:.12f}")
print(f"⏱️ Time Taken (predict only): {(t1-t0):.4f} seconds\n")

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

print("\nClassification Report:")
print(classification_report(y_true, y_pred, digits=6))

# 10) Save predictions
sa_out = sa.copy()
sa_out['pred_label'] = y_pred
try:
    y_proba = model.predict_proba(X_df)
    y_proba = np.array(y_proba)
    if y_proba.ndim == 2 and y_proba.shape[1] == 2:
        sa_out['pred_proba_pos'] = y_proba[:,1]
    else:
        sa_out['pred_proba'] = [list(map(float,row)) for row in y_proba]
except Exception:
    pass

sa_out.to_csv(OUT_CSV, index=False)
print(f"\nSaved predictions to: {OUT_CSV}")


Loaded test file. Shape: (176038, 76)
Model expects 75 features.
Prepared X_df shape: (176038, 75)

Accuracy: 0.008163010259
📌 F1 Score (weighted): 0.000132190401
📌 Recall (weighted): 0.008163010259
📌 Precision (weighted): 0.000066634736
⏱️ Time Taken (predict only): 0.2823 seconds

Confusion Matrix:
[[     0 174601]
 [     0   1437]]

Classification Report:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0   0.000000  0.000000  0.000000    174601
           1   0.008163  1.000000  0.016194      1437

    accuracy                       0.008163    176038
   macro avg   0.004082  0.500000  0.008097    176038
weighted avg   0.000067  0.008163  0.000132    176038


Saved predictions to: /kaggle/working/inital_2017_catboost_ready_with_preds.csv


In [82]:
import pandas as pd
import numpy as np

df = pd.read_csv("/kaggle/working/inital_2017_catboost_ready_binary.csv")

print("Shape:", df.shape)

# Remove label
features = df.drop(columns=["Label"])

# -----------------------------
# 1) Check for constant columns
# -----------------------------
constant_cols = []
zero_cols = []
empty_cols = []

for col in features.columns:
    unique_vals = features[col].dropna().unique()
    if len(unique_vals) == 1:
        constant_cols.append(col)
    if pd.to_numeric(features[col], errors='coerce').fillna(0).eq(0).all():
        zero_cols.append(col)
    if features[col].astype(str).str.strip().eq("").all():
        empty_cols.append(col)

print("\n🔍 CONSTANT columns (single unique value):")
print(constant_cols)

print("\n🔍 ZERO columns (all numeric zeros):")
print(zero_cols)

print("\n🔍 EMPTY STRING columns (possibly IP/FlowID placeholders):")
print(empty_cols)

# --------------------------------
# 2) Summary statistics (numerical)
# --------------------------------
desc = features.describe(include='all').transpose()
desc.to_csv("/kaggle/working/feature_debug_stats.csv")

print("\n📁 Saved numeric summary to: feature_debug_stats.csv")

# --------------------------------
# 3) Show first row to verify actual values
# --------------------------------
print("\n🔍 First row of data:")
print(features.head(1).transpose())


Shape: (176038, 76)

🔍 CONSTANT columns (single unique value):
['Dst Port', 'Timestamp', 'Fwd URG Flags', 'CWE Flag Count', 'Src Port']

🔍 ZERO columns (all numeric zeros):
['Dst Port', 'Timestamp', 'Fwd URG Flags', 'CWE Flag Count', 'Flow ID', 'Src IP', 'Src Port', 'Dst IP']

🔍 EMPTY STRING columns (possibly IP/FlowID placeholders):
[]

📁 Saved numeric summary to: feature_debug_stats.csv

🔍 First row of data:
                         0
Dst Port               0.0
Protocol               6.0
Timestamp              0.0
Flow Duration  112740690.0
Tot Fwd Pkts          32.0
...                    ...
Idle Min        15400000.0
Flow ID                NaN
Src IP                 NaN
Src Port               0.0
Dst IP                 NaN

[75 rows x 1 columns]


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [83]:
import pandas as pd
import numpy as np
from pathlib import Path

path = "/kaggle/working/inital_2017_catboost_ready_binary.csv"
df = pd.read_csv(path)

print("Loaded:", df.shape)

# ------------------------------
# FIX 1 — Dst Port
# ------------------------------
np.random.seed(42)

common_ports = [80, 443, 53, 22]
n = len(df)

df["Dst Port"] = np.where(
    np.random.rand(n) < 0.80,              # 80% common ports
    np.random.choice(common_ports, size=n),
    np.random.randint(1024, 65535, size=n) # 20% random ephemeral
)

# ------------------------------
# FIX 2 — Src Port
# ------------------------------
df["Src Port"] = np.random.randint(1024, 65535, size=n)

# ------------------------------
# FIX 3 — Timestamp (synthetic)
# ------------------------------
avg_flow = df["Flow Duration"].mean()
df["Timestamp"] = np.arange(n) * avg_flow * 0.0001

# ------------------------------
# FIX 4 — Synthetic IPs
# ------------------------------

def random_ip_block(prefix):
    return prefix + "." + str(np.random.randint(1, 255)) + "." + str(np.random.randint(1, 255))

df["Src IP"] = [
    random_ip_block("192.168") for _ in range(n)
]

df["Dst IP"] = [
    random_ip_block("10.0") for _ in range(n)
]

# ------------------------------
# FIX 5 — Fwd URG Flags, CWE Flag Count
# ------------------------------
df["Fwd URG Flags"] = 0      # correct for 99% flows
df["CWE Flag Count"] = 0     # normal for 2018 training data

# ------------------------------
# FIX 6 — Flow ID reconstruction
# ------------------------------
df["Flow ID"] = (
    df["Src IP"].astype(str) + "-" +
    df["Dst IP"].astype(str) + "-" +
    df["Protocol"].astype(str) + "-" +
    df["Src Port"].astype(str) + "-" +
    df["Dst Port"].astype(str)
)

# ------------------------------
# Save cleaned file
# ------------------------------
out = "/kaggle/working/inital_2017_catboost_fixed.csv"
df.to_csv(out, index=False)

print("\n✔ FIXED dataset saved:", out)
print("New shape:", df.shape)
print("\nLabel counts:")
print(df["Label"].value_counts())


Loaded: (176038, 76)

✔ FIXED dataset saved: /kaggle/working/inital_2017_catboost_fixed.csv
New shape: (176038, 76)

Label counts:
Label
0    174601
1      1437
Name: count, dtype: int64


In [84]:
# === Evaluate CatBoost model on provided CSV ===
import os, time, joblib, pickle
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix, classification_report

MODEL_PATH = "/kaggle/working/catboost_ids2018_model.pkl"
TEST_CSV   = "/kaggle/working/inital_2017_catboost_fixed.csv"
OUT_CSV    = "/kaggle/working/inital_2017_catboost_ready_with_preds.csv"
Path(OUT_CSV).parent.mkdir(parents=True, exist_ok=True)

# 1) Load test CSV
if not os.path.exists(TEST_CSV):
    raise FileNotFoundError(f"Test CSV not found: {TEST_CSV}")
sa = pd.read_csv(TEST_CSV)
print("Loaded test file. Shape:", sa.shape)

# 2) Ensure Label exists and create y_true (binary)
if 'Label' not in sa.columns:
    raise ValueError("Test CSV must contain a 'Label' column.")

# Normalize common label variants to binary 0/1
def normalize_label_column(s):
    s2 = s.astype(str).str.strip().str.lower()
    benign_set = {"benign","begnin","normal","0","no","false","n","benign\r"}
    return s2.apply(lambda x: 0 if any(k == x or k in x for k in benign_set) else 1).astype(int)

try:
    # If already numeric 0/1, this keeps them
    if set(sa['Label'].dropna().unique()) <= {0, 1}:
        y_true = sa['Label'].astype(int).to_numpy()
    else:
        y_true = normalize_label_column(sa['Label']).to_numpy()
except Exception:
    y_true = normalize_label_column(sa['Label']).to_numpy()

# 3) Load model (joblib then pickle)
model = None
last_err = None
for loader in (joblib.load, pickle.load):
    try:
        model = loader(MODEL_PATH) if loader is joblib.load else loader(open(MODEL_PATH, 'rb'))
        break
    except Exception as e:
        last_err = e
if model is None:
    raise RuntimeError(f"Failed to load model. Last error: {last_err}")

# 4) Get model feature list
if hasattr(model, "feature_names_"):
    model_features = list(model.feature_names_)
elif hasattr(model, "feature_names"):
    model_features = list(model.feature_names)
else:
    raise RuntimeError("Model does not expose feature names. Provide the model feature list.")

print("Model expects", len(model_features), "features.")

# 5) Prepare X_test: drop Label, add missing features, drop extras, reorder
X_df = sa.drop(columns=['Label']).copy()

missing = [c for c in model_features if c not in X_df.columns]
extras  = [c for c in X_df.columns if c not in model_features]

if missing:
    print("Adding missing model features with sensible defaults:", missing)
    for c in missing:
        if c in ('Src IP','Dst IP','Flow ID'):
            X_df[c] = ""      # string placeholders
        elif c in ('Src Port','Dst Port','Timestamp'):
            X_df[c] = 0       # numeric placeholders
        else:
            X_df[c] = 0       # numeric default

if extras:
    print("Dropping extra columns not used by the model (examples):", extras[:10])
    X_df = X_df.drop(columns=extras)

# Reorder exactly
X_df = X_df[model_features].copy()
print("Prepared X_df shape:", X_df.shape)

# 6) Coerce numeric-like columns (preserve IP/FlowID as strings)
non_numeric_keep = {'Src IP','Dst IP','Flow ID'}
for c in X_df.columns:
    if c in non_numeric_keep:
        X_df[c] = X_df[c].astype(str).fillna("")
    else:
        X_df[c] = pd.to_numeric(X_df[c], errors='coerce').fillna(0)

# 7) Predict (time measured)
t0 = time.time()
try:
    y_pred_raw = model.predict(X_df)
except Exception:
    # try CatBoost Pool fallback
    try:
        from catboost import Pool
        y_pred_raw = model.predict(Pool(X_df))
    except Exception as e:
        raise RuntimeError(f"Prediction failed: {e}")
t1 = time.time()

y_pred = np.array(y_pred_raw).squeeze()
# if model returned probabilities or floats, convert to 0/1
if y_pred.dtype.kind == 'f' and not set(np.unique(y_pred)).issubset({0.0,1.0}):
    # rounding is a safe fallback
    y_pred = np.rint(y_pred).astype(int)
else:
    y_pred = y_pred.astype(int)

# 8) Metrics
acc   = accuracy_score(y_true, y_pred)
f1_w  = f1_score(y_true, y_pred, average='weighted', zero_division=0)
rec_w = recall_score(y_true, y_pred, average='weighted', zero_division=0)
prec_w= precision_score(y_true, y_pred, average='weighted', zero_division=0)

# 9) Print results (clean)
print(f"\nAccuracy: {acc:.12f}")
print(f"📌 F1 Score (weighted): {f1_w:.12f}")
print(f"📌 Recall (weighted): {rec_w:.12f}")
print(f"📌 Precision (weighted): {prec_w:.12f}")
print(f"⏱️ Time Taken (predict only): {(t1-t0):.4f} seconds\n")

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

print("\nClassification Report:")
print(classification_report(y_true, y_pred, digits=6))

# 10) Save predictions
sa_out = sa.copy()
sa_out['pred_label'] = y_pred
try:
    y_proba = model.predict_proba(X_df)
    y_proba = np.array(y_proba)
    if y_proba.ndim == 2 and y_proba.shape[1] == 2:
        sa_out['pred_proba_pos'] = y_proba[:,1]
    else:
        sa_out['pred_proba'] = [list(map(float,row)) for row in y_proba]
except Exception:
    pass

sa_out.to_csv(OUT_CSV, index=False)
print(f"\nSaved predictions to: {OUT_CSV}")


Loaded test file. Shape: (176038, 76)
Model expects 75 features.
Prepared X_df shape: (176038, 75)


RuntimeError: Prediction failed: Bad value for num_feature[non_default_doc_idx=0,feature_idx=71]="192.168.25.13-10.0.232.160-6-61153-443": Cannot convert '192.168.25.13-10.0.232.160-6-61153-443' to float

In [88]:
rr=pd.read_csv("/kaggle/working/2019_cleaned_combined_binary.csv")
rr.columns

Index(['Dst Port', 'Timestamp', 'Flow Duration', 'Fwd Pkt Len Min',
       'Fwd Pkt Len Mean', 'Bwd Pkt Len Max', 'Bwd Pkt Len Min',
       'Bwd Pkt Len Std', 'Flow Pkts/s', 'Fwd IAT Std', 'Fwd Header Len',
       'Fwd Pkts/s', 'Pkt Len Mean', 'Pkt Len Var', 'SYN Flag Cnt',
       'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Subflow Bwd Byts',
       'Init Bwd Win Byts', 'Fwd Seg Size Min', 'Idle Mean', 'Idle Std',
       'Idle Min', 'Src IP', 'Dst IP', 'Label'],
      dtype='object')

In [ ]:
rr.shape